### End Goal:
Using the Nazario_5 dataset from https://zenodo.org/records/8339691, I will build 3 basic classifier models that will be ablt to accurately classify whether a email is a phishing email or not. Then afterwards, I will test the robustness of this model using the TextAttack framework to see how well the model will combat against semantic attacks at small budgets with standard constraints. Then, for each baseline model I'll use a upgraded variation to test how effective different defensive stratgies are for TextAttack.

This will all be evaluted through a report of clean accuracy, robust accuracy vs. edit budget, attack success rate, and testing a couple of defense strategies.

## Problem & Hypothesis:

---

**Problem:** How well can small, targeted changes *(via discrete token edits)* to text inputs can cause a classifier to misidentify whether an email is a phishing attack or not? And are there any effective defensive techniques you can apply to model to combat this?

**Hypothesis:** Against a  baseline encoder-only transformer used to detect phishing emails, I completely believe that an attacker can effective use realistic
perturbation to an email's subject/body text *(via TextAttack)* to fool a classifier into thinking a phishing attack is a legit email with over a 50% success rate at the minimum to an upwards of 90+% success rate.

However, to combat this I've though of a defensive strategy in where the model uses RDAP and DNS lookups as additional features to counteract this.s.

<div style="background:#0f172a; color:#f9fafb; padding:18px; border-radius:10px; border:1px solid #1e293b;">

<h1 style="margin-top:0; font-size:1.9em;">📧 Establishing Email Datasets for the Phishing Classifier</h1>

<p style="margin-bottom:0;">
This notebook prepares all of the <strong>email datasets</strong> used throughout the project.  
We will:
</p>

<ul>
  <li>Standardize multiple legacy email corpora into a <strong>single, unified schema</strong>.</li>
  <li>Create <strong>3-class labels</strong> (ham / spam / phishing) + a binary phishing label.</li>
  <li>Merge <strong>machine-collected datasets</strong> with <strong>human-curated emails</strong>.</li>
  <li>Clean dates, sender domains, and remove cross-dataset duplicates.</li>
  <li>Export the final combined dataset and several specialized splits.</li>
</ul>

</div>

<div style="margin-top:10px; padding:12px; border-left:5px solid #3b82f6; background:#eff6ff;">
  <strong>Step 0 – Global configuration & imports</strong><br>
  We now fix a global random seed and import all libraries needed for:
  <em>data cleaning, feature extraction, domain parsing, model training, and synthetic data generation.</em>
</div>


In [ ]:
!pip install dnspython

In [ ]:
import random
import numpy as np
import torch

SEED = 67  # choose any number you want

random.seed(SEED)
np.random.seed(SEED)

# PyTorch CPU and GPU (if available)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Ensures determinism in certain CUDA operations
torch.use_deterministic_algorithms(True, warn_only=True)

In [ ]:
#Library List
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os, re, ast, json
import requests
import mailbox
import email
import dns
import dns.resolver
import requests

from sklearn.model_selection import train_test_split, cross_validate #Added from stat_quest
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, make_scorer, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay #Added classification report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import OneClassSVM
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from email.header import decode_header
from tqdm.notebook import tqdm

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
from datasets import Dataset
from transformers import Trainer, TrainingArguments, DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.svm import OneClassSVM





## Data Plan and Related Work

---
**Comprehensive data source descriptions:** The dataset I plan to use comes from this publishing from Zenodo, https://zenodo.org/records/8339691, where they haved curated 11 datasets of both legitimate and phishing emails from 1995 to 2022.

Every dataset has the subject text, body text, date, sender/reciever names AND emails, and sometimes even urls as features. Every dataset also comes with a target label identifing whether an email is a phishing attack or not. All in all, it contains everything I need to simulate real-world email environnment.

<h2 style="border-bottom:3px solid #3b82f6; padding-bottom:4px;">
📂 Step 1 – Load all raw email datasets
</h2>

<p>
We now mount Google Drive and load the individual email corpora:
<code>Nazario_5</code>, <code>Nigerian_5</code>, <code>SpamAssassin</code>, <code>TREC_07</code>, and the aggregated
<code>merged_all_emails.csv</code> file.
Each dataset uses a slightly different schema, label convention, and time span, so later steps will normalize them.
</p>

<details>
  <summary><strong>🔎 What to pay attention to here</strong></summary>
  <ul>
    <li>Check that all CSVs load without errors.</li>
    <li>Verify columns like <code>sender</code>, <code>receiver</code>, <code>date</code>, <code>subject</code>, <code>body</code>, <code>label</code>, and <code>urls</code> exist.</li>
    <li>Confirm that <code>PATH</code> points to the correct dataset directory in Drive.</li>
  </ul>
</details>


In [ ]:
#For laptop
#PATH = "/Users/araul/OneDrive - University Of Houston/COSC 4368 - Fundamentals of Artificial Intelligence/Assignment 2 - CNN Design Challenge/Datasets/"

#For desktop
#PATH = "/Users/Ryan/OneDrive - University Of Houston/COSC 4368 - Fundamentals of Artificial Intelligence/Assignment 2 - CNN Design Challenge/Datasets/"

#For Google Colab\
from google.colab import drive
drive.mount("/content/drive")

import os

# === Single project root (EDIT THIS ONLY if you move the folder) ===
PROJECT_ROOT = "/content/drive/MyDrive/COSC 4371 - Security Analytics/2015774/"

# Datasets folder used throughout this notebook
DATA_PATH = os.path.join(PROJECT_ROOT, "Datasets/")
PATH = DATA_PATH  # keep old name so the rest of the notebook still works

# Load Nazario_5 using the unified PATH
Nazario5 = pd.read_csv(os.path.join(PATH, "Nazario_5.csv"))
Nazario5.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr\nPW: bnaweb22\n\n\n ...,0,"['http://web.bna.com', 'http://pubs.bna.com/ip..."
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,[]
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,['http://eastpower.dev.corp.enron.com/summary/...
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald,\n\nWe are currently trading under...",0,[]
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,[]


In [ ]:

#Setting up models folder
import os
import joblib
MODEL_DIR = os.path.join(PATH, "models")
os.makedirs(MODEL_DIR, exist_ok=True)
MODEL_DIR


'/content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/models'

In [ ]:
Nigerian5 = pd.read_csv(PATH + "Nigerian_5.csv")
Nigerian5.head()

,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr PW: bnaweb22 -----O...,0,1
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,0
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,1
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald, We are currently trading under GT...",0,0
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,0


In [ ]:
SpamAssasin = pd.read_csv(PATH + "SpamAssassin.csv")
SpamAssasin.head()

,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


In [ ]:
TREC = pd.read_csv(PATH + "TREC_07.csv")
TREC.head()

,sender,receiver,date,subject,body,label,urls
0,Tomas Jacobs <RickyAmes@aol.com>,the00@speedy.uwaterloo.ca,"Sun, 08 Apr 2007 21:00:48 +0300","Generic Cialis, branded quality@",\n\n\n\n\n\n\nDo you feel the pressure to perf...,1,0
1,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,"Sun, 08 Apr 2007 12:52:30 -0400",Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",0,1
2,Sheila Crenshaw <7stocknews@tractionmarketing....,the00@plg.uwaterloo.ca,"Sun, 08 Apr 2007 17:12:19 +0000",authentic viagra,Mega authenticV I A G R A $ DISCOUNT priceC...,1,1
3,Stormy Dempsey <vqucsmdfgvsg@ruraltek.com>,opt4@speedy.uwaterloo.ca,"Sun, 08 Apr 2007 17:15:47 -0100",Nice talking with ya,"\nHey Billy, \n\nit was really fun going out t...",1,1
4,"""Christi T. Jernigan"" <dcube@totalink.net>",ktwarwic@speedy.uwaterloo.ca,"Sun, 08 Apr 2007 19:19:07 +0200",or trembling; stomach cramps; trouble in sleep...,"\nsystem"" of the home. It will have the capab...",1,0


In [ ]:
df = pd.read_csv(PATH + "merged_all_emails.csv")
print(df.columns)
df.head(10)

Index(['No.', 'Original email No.', 'Subject', 'Body', 'Sender', 'URL(s)',
       'File', 'LLM detected emotion', 'LLM detected motivation', 'Type',
       'Created by', 'Source', 'Year'],
      dtype='object')


,No.,Original email No.,Subject,Body,Sender,URL(s),File,LLM detected emotion,LLM detected motivation,Type,Created by,Source,Year
0,1,1,New login on your pCloud account,Did you recently log in? Someone just tried to...,pCloud Team <noreply@isaccess.co.za>,NaN,NaN,"['Fear', 'Urgency', 'Authority']",['Follow the link'],Phishing,Human,Manual,2025
1,2,2,INVITATION TO THE 79TH IFALPA ANNUAL CONFERENCE,"Dear Anonymous, On behalf of the IFALPA local ...",International Federation Of Air Line Pilots A...,NaN,NaN,"['Urgency', 'Authority', 'Greed']",['Reply'],Phishing,Human,Manual,2025
2,3,3,bitcoin payment,Hello Your Wallet is completed and has been fu...,PERROT Hugo <Hugo.PERROT@SDIS78.FR>,NaN,NaN,"['Greed', 'Curiosity']",['Follow the link'],Phishing,Human,Manual,2025
3,4,4,DHL Megjegyzés: A csomag kiadásához vámkezelé...,Tisztelt DHL Ügyfél! A DHL csomagja kézbesítés...,DHL Package <dhl@shop.dunacademy.info>,https://tinyurl.com/3e6e4yuw,NaN,"['Urgency', 'Fear', 'Authority']",['Follow the link'],Phishing,Human,Manual,2025
4,5,5,Quote Request - Mol Group,"Greetings, Kindly source for the below items a...",Mol Group <supplies@molgr-team.info>,NaN,NaN,"['Urgency', 'Authority']",['Reply'],Phishing,Human,Manual,2025
5,6,6,FEDERAL BUREAU OF INVESTIGATIONS (F.B.I),MR. CHRISTOPHER A. WRAY <itou.kenta@sea.plala....,MR. CHRISTOPHER A. WRAY <itou.kenta@sea.plala....,NaN,NaN,"['Fear', 'Greed', 'Authority', 'Urgency']","['Reply', 'Financial fraud']",Phishing,Human,Manual,2025
6,7,7,Universitetet i Agder har gitt ny informasjon ...,"Hei anonymous@ifi.uio.no, Universitetet i Agde...",noreply@usit.uio.no<noreply-usit.uio.no-735@un...,https://99epicphotosal.za.com/cl6a/,NaN,"['Urgency', 'Authority']",['Follow the link'],Phishing,Human,Manual,2025
7,8,8,New Scanned document from Office Printer,Office PrinterYou received a new scanned docum...,NaN,NaN,NaN,"['Curiosity', 'Urgency']",['Follow the link'],Phishing Simulation,Human,Manual,2025
8,9,9,Welcome to your Corporate Card Account,"CorporateNetDear Anonymous,We provide you and ...",NaN,NaN,NaN,"['Authority', 'Urgency']",['Follow the link'],Phishing Simulation,Human,Manual,2025
9,10,10,Unusual sign-in activity,Microsoft account Unusual sign-in activity We ...,NaN,NaN,NaN,"['Fear', 'Urgency', 'Authority']",['Follow the link'],Phishing Simulation,Human,Manual,2025


<h2 style="border-bottom:3px solid #16a34a; padding-bottom:4px;">
🧍‍♀️🧍 Step 2 – Isolate human-created, non-Nazario emails
</h2>

<p>
From <code>merged_all_emails.csv</code>, we extract a clean subset of
<strong>human-authored emails</strong> that are <em>not</em> part of the Nazario dataset.
This subset will later be merged into the larger combined corpus as an additional source called
<code>MergedAll_Human</code>.
</p>

<div style="padding:10px; background:#dcfce7; border-left:5px solid #15803d; margin-top:6px;">
We also:
<ul>
  <li>Keep only the core fields: <code>sender</code>, <code>receiver</code>, <code>date</code>, <code>subject</code>, <code>body</code>, <code>urls</code>, and <code>label</code>.</li>
  <li>Generate a realistic <code>date</code> column from the <code>Year</code> field using a fixed random seed.</li>
  <li>Ensure a placeholder <code>receiver</code> exists to match the unified schema.</li>
</ul>
</div>


In [ ]:
# --- 1. Filter to human-created, non-Nazario emails from merged_all_email.csv ---

# keep only rows created by a human
mask_human = df["Created by"].str.contains("human", case=False, na=False)

# drop any rows whose Source mentions Nazario (e.g., "Nazario Line ###")
mask_not_nazario = ~df["Source"].str.contains("Nazario", case=False, na=False)

df_human = df[mask_human & mask_not_nazario].copy()
print("Human, non-Nazario emails:", len(df_human))

# --- 2. Build the minimal schema you want ---
# we only keep Subject/Body/Sender/URL(s)/Type/Year
cols_keep = ["Subject", "Body", "Sender", "URL(s)", "Type", "Year"]
df_human = df_human[cols_keep].copy()

# create a proper datetime "date" column from Year
# assign each email a random day in that year using the global SEED
rng = np.random.default_rng(SEED)
df_human["date"] = (
    pd.to_datetime(df_human["Year"].astype(str), format="%Y")
    + pd.to_timedelta(rng.integers(0, 365, size=len(df_human)), unit="D")
)

# drop the Year column (you said you don't want it anymore)
df_human = df_human.drop(columns=["Year"])

# --- 3. Rename columns to match your combined dataset schema ---
df_human = df_human.rename(
    columns={
        "Subject": "subject",
        "Body": "body",
        "Sender": "sender",
        "URL(s)": "urls",
        "Type": "label",  # keep "Type" as your label for now
    }
)

# your normalization code expects a 'receiver' column, so add a placeholder
df_human["receiver"] = "[UNKNOWN]"

# reorder columns to align with the combined dataset
df_human = df_human[["sender", "receiver", "date", "subject", "body", "label", "urls"]]

print(df_human.head())


Human, non-Nazario emails: 2058
                                              sender   receiver       date  \
0               pCloud Team <noreply@isaccess.co.za>  [UNKNOWN] 2025-10-13   
1   International Federation Of Air Line Pilots A...  [UNKNOWN] 2025-06-01   
2                PERROT Hugo <Hugo.PERROT@SDIS78.FR>  [UNKNOWN] 2025-07-24   
3             DHL Package <dhl@shop.dunacademy.info>  [UNKNOWN] 2025-02-16   
4               Mol Group <supplies@molgr-team.info>  [UNKNOWN] 2025-06-12   

                                             subject  \
0                   New login on your pCloud account   
1    INVITATION TO THE 79TH IFALPA ANNUAL CONFERENCE   
2                                    bitcoin payment   
3   DHL Megjegyzés: A csomag kiadásához vámkezelé...   
4                          Quote Request - Mol Group   

                                                body     label  \
0  Did you recently log in? Someone just tried to...  Phishing   
1  Dear Anonymous, On behalf o

<h2 style="border-bottom:3px solid #a855f7; padding-bottom:4px;">
🧰 Step 3 – Shared normalization utilities & dataset config
</h2>

<p>
Next we define reusable helper functions that turn each raw dataset into a
<strong>common email schema</strong>:
</p>

<ul>
  <li>Robust URL extraction from plain text, lists, or HTML.</li>
  <li>Email parsing to obtain <code>sender_email</code> and <code>sender_domain</code>.</li>
  <li>Safe date parsing into timezone-naive <code>datetime</code> objects.</li>
  <li>Placeholder filling for missing <code>subject</code> / <code>body</code>.</li>
</ul>

<p>
We also create a <code>DATASETS</code> configuration list that records, for each source
(<code>Nazario_5</code>, <code>Nigerian_5</code>, <code>SpamAssassin</code>, <code>TREC</code>),
its filename and label scheme (phishing vs spam).
This metadata drives the next step where we load, normalize, and relabel each dataset consistently.
</p>


In [ ]:
#Step 1 – Helper functions (normalize + domain extraction)

EMAIL_RE = re.compile(r'[\w\.-]+@[\w\.-]+')
HTTP_RE = re.compile(r'https?://\S+')

URL_RE = re.compile(
    r'(?i)\b('
    r'(?:https?://|www\.|ftp://|hxxp://)'   # protocol or pseudo-protocol
    r'[^\s<>"\']+'                          # domain + path
    r')'
)

def coerce_urls_strict(x):
    """Extract URLs from literal lists, plain text, HTML, or obfuscated patterns."""
    if pd.isna(x):
        return []

    # already a list?
    if isinstance(x, list):
        return [str(u).strip().rstrip('.,);]>"\'') for u in x if str(u).strip()]

    s = str(x)

    # Try literal list first
    try:
        val = ast.literal_eval(s)
        if isinstance(val, list):
            return [str(u).strip().rstrip('.,);]>"\'') for u in val if str(u).strip()]
    except:
        pass

    # Fallback: full regex extraction
    found = re.findall(URL_RE, s)
    cleaned = [u.strip().rstrip('.,);]>"\'') for u in found]
    return cleaned

def extract_emails(text):
    if pd.isna(text):
        return []
    return EMAIL_RE.findall(str(text))

def extract_domains_from_emails(emails):
    out = []
    for e in emails:
        try:
            out.append(e.split('@', 1)[1].lower().strip('>"/>').strip())
        except Exception:
            pass
    return out

def extract_domains_from_urls(urls):
    out = []
    for u in urls:
        try:
            no_scheme = re.sub(r'^https?://', '', u)
            domain = no_scheme.split('/')[0].lower().split(':')[0]
            out.append(domain)
        except Exception:
            pass
    return out

def ensure_columns(df, cols):
    """Make sure all expected columns exist; if missing, create as NA."""
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df

def normalize_email_frame(df, source_name):
    """
    Normalize a single dataset to a common email schema, but do NOT
    touch labels here. We'll map labels separately.
    """
    expected = ["sender", "receiver", "date", "subject", "body", "label", "urls"]
    df = ensure_columns(df, expected)

    # Basic types
    for c in ["sender", "receiver", "subject", "body", "date"]:
        df[c] = df[c].astype("string")

    # URLs → list
    df["urls"] = df["urls"].apply(coerce_urls_strict)

    # Robust date parsing, make timezone-naive
    dt = pd.to_datetime(df["date"], errors="coerce", utc=True)
    df["date"] = dt.dt.tz_convert(None)

    # Emails & domains
    df["sender_emails"] = df["sender"].apply(extract_emails)
    df["receiver_emails"] = df["receiver"].apply(extract_emails)

    df["sender_email"] = df["sender_emails"].apply(lambda xs: xs[0].lower() if xs else pd.NA)
    df["receiver_email_primary"] = df["receiver_emails"].apply(lambda xs: xs[0].lower() if xs else pd.NA)

    df["sender_domain"] = df["sender_emails"].apply(
        lambda xs: xs[0].split("@", 1)[1].lower() if xs else pd.NA
    )

    df["receiver_domains"] = df["receiver_emails"].apply(extract_domains_from_emails)
    df["url_domains"] = df["urls"].apply(extract_domains_from_urls)

    # Fill missing subject/body with placeholders
    df["subject"] = df["subject"].fillna("[NO_SUBJECT]")
    df["body"] = df["body"].fillna("[NO_BODY]")

    # Tag dataset of origin
    df["source_dataset"] = source_name

    return df

In [ ]:
DATASETS = [
    {
        "name": "Nazario_5",
        "filename": "Nazario_5.csv",
        "label_scheme": "phishing",  # 0=ham, 1=phish
    },
    {
        "name": "Nigerian_5",
        "filename": "Nigerian_5.csv",
        "label_scheme": "phishing",  # 0=ham, 1=phish
    },
    {
        "name": "SpamAssasin",
        "filename": "SpamAssassin.csv",
        "label_scheme": "spam",      # 0=ham, 1=spam
    },
    {
        "name": "TREC",
        "filename": "TREC_07.csv",
        "label_scheme": "spam",      # 0=ham, 1=spam (assumed)
    },
]


<h2 style="border-bottom:3px solid #f97316; padding-bottom:4px;">
🧪 Step 4 – Map labels to 3-class scheme & stack machine datasets
</h2>

<p>
Here we:
</p>

<ol>
  <li><strong>Normalize</strong> each dataset using <code>normalize_email_frame</code>.</li>
  <li><strong>Map</strong> original labels into a unified 3-class scheme:<br>
      <code>0 = ham_legit</code>, <code>1 = spam</code>, <code>2 = phishing</code>.
  </li>
  <li>Apply <strong>TREC-specific downsampling</strong> so that massive TREC spam/ham
      doesn’t dominate the combined corpus.</li>
  <li>Collect all normalized frames into a single list <code>frames</code> for concatenation.</li>
</ol>

<details>
  <summary><strong>⚖️ Why 3 classes?</strong></summary>
  <p>
  Distinguishing <em>spam</em> from <em>targeted phishing</em> lets us evaluate models on both
  generic unsolicited junk and high-risk credential theft emails, while still supporting
  a simpler binary phishing vs non-phishing view.
  </p>
</details>


In [ ]:
#Step 3 – Load each dataset, map to 3-class labels individually
def map_labels_3class(df, label_scheme):
    """
    Create label_3class from the dataset's original label + scheme.
    0 = ham, 1 = spam, 2 = phishing
    """
    if label_scheme == "phishing":
        # 0 = ham, 1 = phishing
        df["label_3class"] = df["label"].map({0: 0, 1: 2})

    elif label_scheme == "all_phishing":
        df["label_3class"] = 2

    elif label_scheme == "spam":
        # 0 = ham, 1 = spam
        df["label_3class"] = df["label"].map({0: 0, 1: 1})

    else:
        # You can add other schemes later if needed
        raise ValueError(f"Unknown label_scheme: {label_scheme}")

    return df


<h2 style="border-bottom:3px solid #6366f1; padding-bottom:4px;">
🧬 Step 5 – Merge all sources & remove cross-dataset duplicates
</h2>

<p>
We now concatenate all normalized machine datasets into a single DataFrame,
then append the human-created subset as <code>MergedAll_Human</code>.
</p>

<div style="padding:10px; background:#eef2ff; border-left:5px solid #4f46e5; margin-top:6px;">
<strong>De-duplication strategy</strong><br>
We construct a <code>text_sig</code> from <code>subject || body</code> and:
<ul>
  <li>Sort by a <code>source_priority</code> so preferred datasets win tie-breaks.</li>
  <li>Drop duplicate <code>text_sig</code> across <em>all</em> sources.</li>
</ul>
This prevents the same email from appearing multiple times under different dataset names.
</div>


In [ ]:
# Step 4 – Loop over configs, normalize + relabel + combine
frames = []

for cfg in DATASETS:
    src_name = cfg["name"]
    filename = cfg["filename"]
    scheme = cfg["label_scheme"]

    path = os.path.join(PATH, filename)
    df_raw = pd.read_csv(path)

    # normalize columns and extract domains, etc.
    df_norm = normalize_email_frame(df_raw, src_name)

    # map labels to 3-class
    df_norm = map_labels_3class(df_norm, scheme)

    # ------------------------------
    # TREC-specific downsampling (same as old CEAS logic)
    # ------------------------------
    if src_name == "TREC":
        # separate ham and spam in TREC
        trec_ham = df_norm[df_norm["label_3class"] == 0]
        trec_spam = df_norm[df_norm["label_3class"] == 1]

        # choose fractions to keep (same 10% each, or adjust)
        ham_frac = 0.1
        spam_frac = 0.1

        trec_ham_down = trec_ham.sample(frac=ham_frac, random_state=SEED)
        trec_spam_down = trec_spam.sample(frac=spam_frac, random_state=SEED)

        # recombine downsampled TREC
        df_norm = pd.concat([trec_ham_down, trec_spam_down], ignore_index=True)
        print(
            f"After downsampling TREC: {len(df_norm)} rows "
            f"(ham={len(trec_ham_down)}, spam={len(trec_spam_down)})"
        )

    # add the (possibly downsampled) dataset to frames
    frames.append(df_norm)

# ========================
# Combine all machine datasets
# ========================
combined = pd.concat(frames, ignore_index=True, sort=False)

# --- clean date/year (only if date exists) ---
if "date" in combined.columns:
    combined["date"] = pd.to_datetime(combined["date"], errors="coerce")
    combined["year"] = combined["date"].dt.year
else:
    combined["date"] = pd.NaT
    combined["year"] = pd.NA

# optional: filter to desired years
combined = combined[combined["year"] >= 2000].copy()

# binary label from 3-class (phishing vs not)
combined["label_binary"] = (combined["label_3class"] == 2).astype("int64")

unused_cols = [
    "receiver_email_primary",
    "receiver_domains",
]

# Drop only those columns that exist in the dataset
combined = combined.drop(columns=[c for c in unused_cols if c in combined.columns])

print("Remaining columns after machine datasets:", combined.columns.tolist())

# ==========================================
# Normalize df_human and append to combined
# ==========================================

# df_human should already exist from earlier cells
# and should already have subject/body/sender/urls/date/label_3class, etc.
df_human_norm = normalize_email_frame(df_human, "MergedAll_Human")

# Concatenate with existing combined dataset
combined_extended = pd.concat(
    [combined, df_human_norm],
    ignore_index=True,
    sort=False,
)

print("Original combined rows (machine only):", len(combined))
print("Extended combined rows (with human):", len(combined_extended))

# This will now include the human-created emails properly normalized
combined = combined_extended

# --- Ensure urls column is always a list (optional but safe) ---
def normalize_urls(x):
    if isinstance(x, list):
        # convert all elements to string and drop empty strings
        xs = [str(u).strip() for u in x if str(u).strip()]
        return xs
    if pd.isna(x) or x in ("", "[]"):
        return []
    # single string → wrap in a list
    return [str(x).strip()]

# If urls column is missing for some reason, create an empty-list column
if "urls" not in combined.columns:
    combined["urls"] = [[] for _ in range(len(combined))]

combined["urls"] = combined["urls"].apply(normalize_urls)

# ==========================================
# Remove dataset overlap by subject+body text
# ==========================================

# 1) Build a text signature from subject + body
combined["text_sig"] = (
    combined["subject"].fillna("").str.strip() + "||" +
    combined["body"].fillna("").str.strip()
).str.lower()

# 2) Choose which source to keep when duplicates exist
#    Lower 'source_priority' = preferred row when duplicates exist.
source_priority = {
    "MergedAll_Human": 0,
    "Nazario_5": 1,
    "Nigerian_5": 2,
    "SpamAssasin": 3,
    "TREC": 4,
}
combined["source_priority"] = combined["source_dataset"].map(source_priority).fillna(99)

# 🔴 NEW: save a copy BEFORE we drop duplicates (this keeps all duplicates)
combined_before_dedupe = combined.copy()

before = len(combined)

combined = (
    combined
    # ensure preferred source kept when same text appears in multiple datasets
    .sort_values(["text_sig", "source_priority"])
    # drop exact text duplicates across ALL datasets
    .drop_duplicates(subset=["text_sig"], keep="first")
    .drop(columns=["text_sig", "source_priority"])
    .reset_index(drop=True)
)

after = len(combined)
print(f"Removed {before - after} overlapping emails based on subject+body text")
print("Final rows:", after)


After downsampling TREC: 5376 rows (ham=2436, spam=2940)
Remaining columns after machine datasets: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls', 'sender_emails', 'receiver_emails', 'sender_email', 'sender_domain', 'url_domains', 'source_dataset', 'label_3class', 'year', 'label_binary']
Original combined rows (machine only): 17053
Extended combined rows (with human): 19111
Removed 1517 overlapping emails based on subject+body text
Final rows: 17594


<h2 style="border-bottom:3px solid #e11d48; padding-bottom:4px;">
🩺 Step 6 – Clean dates & enforce valid sender domains
</h2>

<p>
At this stage we enforce basic <strong>temporal and address sanity checks</strong>:
</p>

<ul>
  <li>Parse messy email <code>date</code> strings using a <code>safe_parse_date</code> helper.</li>
  <li>Drop rows with impossible years (before 1990 or after 2025).</li>
  <li>Require non-null, well-formed <code>sender_email</code> and <code>sender_domain</code>.</li>
</ul>

<div style="padding:10px; background:#fee2e2; border-left:5px solid #b91c1c; margin-top:6px;">
These filters trade a small amount of data for <strong>much higher reliability</strong> when we later
compute domain features and RDAP/DNS lookups.
</div>


In [ ]:
# ==========================================
# Fix impossible dates with a safe parser
# ==========================================

from dateutil import parser
import numpy as np
import pandas as pd

def safe_parse_date(value):
    """
    Safely parse email dates:
    - Try dateutil parser
    - Reject impossible years (<1990 or >2025)
    - Return NaT for anything invalid
    """
    if pd.isna(value):
        return pd.NaT

    try:
        dt = parser.parse(str(value), fuzzy=True)

        # Reject impossible years
        if dt.year < 1990 or dt.year > 2025:
            return pd.NaT

        return dt
    except:
        return pd.NaT


print("Cleaning date column...")
combined["date"] = combined["date"].apply(safe_parse_date)
combined["year"] = combined["date"].dt.year

print("Remaining NaT dates after cleaning:", combined["date"].isna().sum())



Cleaning date column...
Remaining NaT dates after cleaning: 6


In [ ]:
# ==========================================
# Fallback for rows where date parsing failed
# ==========================================

mask_missing_date = combined["date"].isna() & combined["year"].notna()

combined.loc[mask_missing_date, "date"] = pd.to_datetime(
    combined.loc[mask_missing_date, "year"].astype(int).astype(str) + "-07-01"
)

# After fixing dates, recompute year
combined["year"] = combined["date"].dt.year

# Remove rows with truly impossible years
mask_bad_year = (combined["year"] < 1990) | (combined["year"] > 2025) | (combined["year"].isna())
removed_bad = mask_bad_year.sum()

combined = combined[~mask_bad_year].copy()

print(f"Removed {removed_bad} rows with impossible or corrupt years.")
print("New row count:", len(combined))


Removed 6 rows with impossible or corrupt years.
New row count: 17588


In [ ]:
# ==========================================
# Require valid sender_email & sender_domain
# ==========================================

before = len(combined)

# 1) basic non-null filter
mask_non_null = combined["sender_email"].notna() & combined["sender_domain"].notna()

combined = combined[mask_non_null].copy()

print("Dropped rows with missing sender_email/domain:", before - len(combined))
print("Rows left after non-null filter:", len(combined))


Dropped rows with missing sender_email/domain: 168
Rows left after non-null filter: 17420


In [ ]:
# Check how many MergedAll_Human emails share identical subject+body text
mask_mh = combined["source_dataset"] == "MergedAll_Human"
mh = combined[mask_mh].copy()

mh["mh_text_sig"] = (
    mh["subject"].fillna("").str.strip().str.lower() + "||" +
    mh["body"].fillna("").str.strip().str.lower()
)

dup_mh = mh.duplicated(subset="mh_text_sig", keep=False)
print("MergedAll_Human rows:", len(mh))
print("MergedAll_Human exact text duplicates (subject+body):", dup_mh.sum())


MergedAll_Human rows: 1934
MergedAll_Human exact text duplicates (subject+body): 0


In [ ]:
import re

email_pattern = re.compile(r"^[^@\s]+@[^@\s]+\.[^@\s]+$")

def is_valid_email(x):
    if pd.isna(x):
        return False
    s = str(x).strip()
    return bool(email_pattern.match(s))

before = len(combined)

mask_valid_email = combined["sender_email"].apply(is_valid_email)
combined = combined[mask_valid_email].copy()

print("Dropped rows with malformed sender_email:", before - len(combined))
print("Rows left after email-format filter:", len(combined))


Dropped rows with malformed sender_email: 13
Rows left after email-format filter: 17407


In [ ]:
# --- Map string labels from MergedAll_Human into 3-class labels ---

# Only rows from merged_all_emails.csv
mask_new = combined["source_dataset"] == "MergedAll_Human"

def map_type_to_3class(val):
    """
    Map the 'Type' / label string from merged_all_emails
    into your 3-class scheme: 0=ham_legit, 1=spam, 2=phishing.

    If an unknown string is seen, raise an error instead of
    silently writing NaN.
    """
    if pd.isna(val):
        raise ValueError("MergedAll_Human row has missing Type/label")

    s = str(val).strip().lower()

    # ham / legitimate
    if "valid" in s or "legit" in s or "ham" in s:
        return 0

    # phishing (includes "phishing simulation")
    if "phishing" in s:
        return 2

    # spam / malicious
    if "spam" in s or "malicious" in s:
        return 1

    # anything else is unexpected – better to crash here
    raise ValueError(f"Unrecognized MergedAll_Human label string: {val!r}")

# Make sure label_3class can hold ints cleanly
combined["label_3class"] = combined["label_3class"].astype("Int64")

# Apply mapping JUST to the MergedAll_Human rows
combined.loc[mask_new, "label_3class"] = (
    combined.loc[mask_new, "label"].apply(map_type_to_3class).astype("Int64")
)

# Quick sanity check: there should be no NaNs in label_3class for these rows
num_nan = combined.loc[mask_new, "label_3class"].isna().sum()
print("MergedAll_Human rows with NaN label_3class:", num_nan)

# Recompute binary label for *all* rows, now that everything is mapped
combined["label_binary"] = (combined["label_3class"] == 2).astype("int64")


MergedAll_Human rows with NaN label_3class: 0


<h2 style="border-bottom:3px solid #0ea5e9; padding-bottom:4px;">
🧮 Step 7 – Train/test split with group-based de-duplication
</h2>

<p>
Instead of splitting rows independently, we build a <code>group_sig</code> based on
<code>subject || body</code> and use <code>GroupShuffleSplit</code> so that
<strong>near-duplicate emails never appear in both train and test</strong>.
</p>

<ul>
  <li>Each unique <code>group_sig</code> gets a numeric <code>group_id</code>.</li>
  <li>The grouped splitter respects both label distribution and text groups.</li>
  <li>We save <code>train_idx.npy</code> and <code>test_idx.npy</code> for reuse in all pipelines.</li>
</ul>

<div style="padding:10px; background:#e0f2fe; border-left:5px solid #0284c7; margin-top:6px;">
This prevents overly optimistic evaluation due to text copies or tiny variants leaking between splits.
</div>


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

print("Building grouped train/test split to avoid near-duplicate leakage...")

# 1. Text signature to group duplicates/near-dups
combined["group_sig"] = (
    combined["subject"].fillna("").str.lower().str.strip() + "||" +
    combined["body"].fillna("").str.lower().str.strip()
)

combined["group_id"] = combined["group_sig"].astype("category").cat.codes

groups = combined["group_id"]
labels = combined["label_3class"].values
indices = np.arange(len(combined))

gss = GroupShuffleSplit(test_size=0.20, n_splits=1, random_state=SEED)
train_idx, test_idx = next(gss.split(indices, labels, groups))

print("Train size:", len(train_idx))
print("Test size:", len(test_idx))

# Save indices for all pipeline notebooks
np.save(os.path.join(PATH, "train_idx.npy"), train_idx)
np.save(os.path.join(PATH, "test_idx.npy"), test_idx)

# Remove helper columns before saving dataset
combined = combined.drop(columns=["group_sig", "group_id"])


Building grouped train/test split to avoid near-duplicate leakage...
Train size: 13925
Test size: 3482


<h2 style="border-bottom:3px solid #22c55e; padding-bottom:4px;">
📊 Step 8 – Inspect label distribution & save master dataset
</h2>

<p>
We now generate a quick summary of the combined corpus:
</p>

<ul>
  <li>Map <code>label_3class</code> into human-readable <code>label_name</code>.</li>
  <li>Count ham / spam / phishing emails per <code>source_dataset</code>.</li>
  <li>Add simple text length features: <code>subject_len</code>, <code>body_len</code>.</li>
  <li>Export the final unified dataset to
      <code>combined_emails_3class.csv</code> (and Parquet when possible).</li>
</ul>

<div style="padding:10px; background:#dcfce7; border-left:5px solid #16a34a; margin-top:6px;">
This file is the <strong>canonical dataset</strong> consumed by all later notebooks:
baseline pipelines, deep pipelines, robustness analysis, and embedding comparisons.
</div>


In [ ]:
# ============================================================
# Step 8 – Inspect label distribution & save master dataset
# + export all post-2010 ham emails
# ============================================================

# 1) Extract ALL post-2010 ham emails from the *cleaned* dataset
#    (this includes MergedAll_Human with proper 3-class labels)
post2010_ham_all = combined.copy()

# Ensure year column exists
post2010_ham_all["year"] = post2010_ham_all["date"].dt.year

# Keep only post-2010 ham (label_3class == 0)
mask_post2010 = post2010_ham_all["year"] >= 2010
mask_ham = post2010_ham_all["label_3class"] == 0
post2010_ham_all = post2010_ham_all[mask_post2010 & mask_ham].copy()
print("Rows after year>=2010 & ham filter:", len(post2010_ham_all))

# Drop ONLY rows missing key fields needed by pipelines
key_cols = ["sender", "sender_email", "sender_domain", "subject", "body", "date"]
post2010_ham_all = post2010_ham_all.dropna(subset=key_cols)
print("Rows after dropping missing key fields:", len(post2010_ham_all))

print("\nMissing values per key column:")
print(post2010_ham_all[key_cols].isna().sum())

print("\nPost-2010 ham by source_dataset:")
print(
    post2010_ham_all
        .groupby("source_dataset")["label_3class"]
        .value_counts()
)

# Save final version
out_path = os.path.join(PATH, "emails_post2010_ham_all.csv")
post2010_ham_all.to_csv(out_path, index=False)
print("\nSaved post-2010 ham emails to:", out_path)

# ============================================================
# Standard master-dataset exports (unchanged logic)
# ============================================================

combined["label_binary"] = (combined["label_3class"] == 2).astype(int)

# Add simple length features before saving
combined["subject_len"] = combined["subject"].fillna("").str.len()
combined["body_len"] = combined["body"].fillna("").str.len()

# save outputs
OUT_CSV = os.path.join(PATH, "combined_emails_3class.csv")
OUT_PARQUET = os.path.join(PATH, "combined_emails_3class.parquet")

combined.to_csv(OUT_CSV, index=False)
try:
    combined.to_parquet(OUT_PARQUET, index=False)
except Exception as e:
    print("Parquet not written:", e)

print("Rows:", len(combined))
print("3-class distribution:\n", combined["label_3class"].value_counts().sort_index())
print("Binary distribution:\n", combined["label_binary"].value_counts().sort_index())


Rows after year>=2010 & ham filter: 70
Rows after dropping missing key fields: 70

Missing values per key column:
sender           0
sender_email     0
sender_domain    0
subject          0
body             0
date             0
dtype: int64

Post-2010 ham by source_dataset:
source_dataset   label_3class
MergedAll_Human  0               70
Name: count, dtype: Int64

Saved post-2010 ham emails to: /content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/emails_post2010_ham_all.csv
Parquet not written: ("Could not convert 'Spam' with type str: tried to convert to int64", 'Conversion failed for column label with type object')
Rows: 17407
3-class distribution:
 label_3class
0    8509
1    5161
2    3737
Name: count, dtype: Int64
Binary distribution:
 label_binary
0    13670
1     3737
Name: count, dtype: int64


In [ ]:
combined["label_binary"] = (combined["label_3class"] == 2).astype(int)

# Add simple length features before saving
combined["subject_len"] = combined["subject"].fillna("").str.len()
combined["body_len"] = combined["body"].fillna("").str.len()


In [ ]:
# save outputs
OUT_CSV = os.path.join(PATH, "combined_emails_3class.csv")
OUT_PARQUET = os.path.join(PATH, "combined_emails_3class.parquet")

combined.to_csv(OUT_CSV, index=False)
try:
    combined.to_parquet(OUT_PARQUET, index=False)
except Exception as e:
    print("Parquet not written:", e)

print("Rows:", len(combined))
print("3-class distribution:\n", combined["label_3class"].value_counts().sort_index())
print("Binary distribution:\n", combined["label_binary"].value_counts().sort_index())

Parquet not written: ("Could not convert 'Spam' with type str: tried to convert to int64", 'Conversion failed for column label with type object')
Rows: 17407
3-class distribution:
 label_3class
0    8509
1    5161
2    3737
Name: count, dtype: Int64
Binary distribution:
 label_binary
0    13670
1     3737
Name: count, dtype: int64


**Demonstrated significance to security analytics:** There are 2 main reasons as to why I specifically chose this dataset...
1.
The first is that is was one of the few datasets that included emails in a somewhat relevant timezone as it even has some emails records dating from 2022. While my preferred date range was somewhere between 2020 - 2025, I couldn't find a more recent email dataset as this was the only reliable (and free) dataset I could get my hands on that had even had records of a 2020+ timestamp. Whether this was because of COVID or just the security issues of obtaining that dataset I'm not sure, but I plan to only use the "relevant" records when I clean the dat
2. .
The secain reason I chose this dataset was how they formatted the features of each email (i.e, subject, body, sender, reciever, date, etc.). This was important to me as not only did I need the timestamps of each email to ensure relevancy but I also needed the domains involved with each email so I can perform RDAP and DNS look for my defensive strategy against TextAttack

With this dataset, through the reasons I provided I'll be able to obtain an email dataset that will sufficiently simulate real-world email samples by ensuring that the dataset is at least somewhat relevant to this point in time and has the provided information to analyze suspicious domain and RDAP behavior in regards to phishing attacks and to use that for classification models.ps.

**Awareness of state-of-the-art / Substantive literature review with proper citations:** Some papers I've read that influenced the idea of my project include...

1. *Hao, S., Kantchelian, A., Miller, B., Paxson, V., & Feamster, N. (2016). PREDATOR: Proactive recognition and elimination of domain abuse at time-of-registration. In Proceedings of the 2016 ACM SIGSAC Conference on Computer and Communications Security (CCS’16) (pp. 1568–1579). Association for Computing Machinery*; https://www2.cs.uh.edu/~gnawali/courses/cosc6377-f23/papers/Hao16.pdf
    - This paper was mentioned to me by my Intro to Computer Networks paper and it was the initial reason I thought of using RDAP and DNS lookups for my transformer. Within this paper it mentions how they developed PREDATOR, an approach that uses only time-of-registration features to establish domain reputation" (Hao, Kantchelian, Miller, Paxson, & Feamster, 2016) and through this PREDATOR achieed a 70% detection rate with a false positive rate of 0.35%, thus proving the idea of utilizing domain behavior as a feature for identifying websites.


2. *Mendes, P., Maia, E., & Praça, I. (2025). MeAJOR Corpus: A multi-source dataset for phishing email detection*; https://arxiv.org/abs/2507.17978    - This paper helped me with my emphasis on dataset relevancy and consistency, especially in phishing research. Similar to how Professor Verma explained in previous lectures, it explains how "addresses common issues in phishing research—such as class imbalance, outdated samples, and inconsistent preprocessing—by applying a unified cleaning and anonymization pipeline.". Thus leads to many of these models to result in unrealistic accuracy. Following their approach, I plan to create a structured and clean dataset that ensures relevancy by using no email older than 2015, properly cleaning the dataset from duplicates, missing values, and domain info. The paper also provided a table list of datasets they used which provides some confidence to the dataset I obtained from Zenodo as both have some overlap in both the data and features they use.1

**Awareness of state-of-the-art:**

3. *Maneriker, P., Stokes, J. W., Garcia Lazo, E., Carutasu, D., Tajaddodianfar, F., & Gururajan, A. (2021). URLTran: Improving phishing URL detection using transformers*; https://arxiv.org/abs/2106.05256
    - This paper was special in that they used models based on BERT and RoBERTa architectures to classify malicious URLs. In fact, their best model "URLTran_BERT, achieved a true positive rate of 86.8% at only 0.01% false positives" not only proving the use of using URLs as feature will be an effective strategy but if I believe if I follow this paper I can implement and additional defensive strategy, apart from the RDAP/DNS lookups, utilize the URLs in an emmail as some datasets have already preprocessed the URLs as their own features.

## <span style="color:red"> Methodology </span>

---
### Exploratory Data Analysis
**Objective**: Conduct a thorough exploratory analysis to first uncover underlying problems within the dataset that we will have to fix during the data pre-processing stage. Then, when the data has been preprocessed engage in a secondary EDA to visualize underlying patterns and insights

**Task**: There will be two major problems I will have to solve for my dataset:
1. Since my dataset is a collection of email features, the majority if not all the data will be represent via text and thus I will have to normalize all text records into a format suitable for my model.
2. If I wish to truly utilize the RDAP and DNS information I can obtain from the domains involved within an email I will also have to create additional features of this information myself.

### Before pre-processing:

<h2 style="border-bottom:3px solid #64748b; padding-bottom:4px;">
🔍 Step 9 – Sanity checks & exploratory data quality analysis
</h2>

<p>
Before creating specialized splits, we run a series of sanity checks on
<code>combined_emails_3class</code>:
</p>

<ul>
  <li>Overall shape (rows × columns).</li>
  <li>Label counts and consistency with original schemes.</li>
  <li>Duplicate detection on key fields.</li>
  <li>URL column format (lists vs scalars).</li>
  <li>Date ranges and impossible timestamps.</li>
</ul>

<details>
  <summary><strong>💡 Why this matters</strong></summary>
  <p>
  Any remaining structural issues here would propagate into every model,
  so this is the last checkpoint to catch weird edge cases before training.
  </p>
</details>


**Check for Dataset Proportions**

In [ ]:
#Checking how big each dataset is
print("The Combined dataset has", combined.shape[0],"instances (rows) and", combined.shape[1], "attributes (columns)")
combined.shape

The Combined dataset has 17407 instances (rows) and 23 attributes (columns)


(17407, 23)

In [ ]:
combined.head()

,sender,receiver,date,subject,body,label,urls,sender_emails,receiver_emails,sender_email,...,label_3class,year,label_binary,receiver_email_primary,receiver_domains,label_name,subject_len,body_len,subject_body,body_only
0,barrister vincent usman <2003vincent@freesurf.fr>,2003vincent@freesurf.fr,2003-08-25 23:14:46,!!!ASSISTANCE NEEDED!!!,"Attention: Friend, I am Barrister Vincent Usm...",1,[],[2003vincent@freesurf.fr],[2003vincent@freesurf.fr],2003vincent@freesurf.fr,...,2,2003.0,1,NaN,NaN,phishing,23,2306,"!!!ASSISTANCE NEEDED!!! Attention: Friend, I ...","Attention: Friend,\n\nI am Barrister Vincent U..."
1,"""Monkey.org-Notification"" <esther@grahamcarrmw...",<jose@monkey.org>,2022-07-19 17:32:53,!Alert jose@monkey.org Passsword Expire.,"monkey.org WEBMAIL Hello jose, Your jose@mon...",1,[],[esther@grahamcarrmw.com],[jose@monkey.org],esther@grahamcarrmw.com,...,2,2022.0,1,NaN,NaN,phishing,40,525,!Alert jose@monkey.org Passsword Expire. monke...,"monkey.org WEBMAIL Hello jose, Your jose@mon..."
2,ethan4576@excite.com,paula9485@excite.com,2002-07-19 19:04:47,!CLICK HERE! How to legally access criminal an...,Useful for your Individual and Business invest...,1,[],[ethan4576@excite.com],[paula9485@excite.com],ethan4576@excite.com,...,1,2002.0,0,NaN,NaN,spam,75,1801,!CLICK HERE! How to legally access criminal an...,Useful for your Individual and Business invest...
3,km523876@caramail.com,<zzzz4@spamassassin.taint.org>,2002-09-05 07:42:34,"!Gorgeous,Custom Websites - $349 Complete! (90...","**Beautiful, 100% Custom Websites, $349 Comp...",1,[],[km523876@caramail.com],[zzzz4@spamassassin.taint.org],km523876@caramail.com,...,1,2002.0,0,NaN,NaN,spam,67,1334,"!Gorgeous,Custom Websites - $349 Complete! (90...","\n\n**Beautiful, 100% Custom Websites, $349 Co..."
4,LIUYANMAILPOST <liuyan_office@yahoo.com.hk>,<NA>,2007-08-28 22:06:15,!WORK WITH ME URGENT!,"Dear friend, This letter must surprise you...",1,[],[liuyan_office@yahoo.com.hk],[],liuyan_office@yahoo.com.hk,...,2,2007.0,1,NaN,NaN,phishing,21,3383,"!WORK WITH ME URGENT! Dear friend, This le...","\n\nDear friend,\n\nThis letter must surprise..."


**Check for correct Data labels**

In [ ]:
print("label_3class value counts:")
print(combined["label_3class"].value_counts(dropna=False).sort_index())


label_3class value counts:
label_3class
0    8509
1    5161
2    3737
Name: count, dtype: Int64


In [ ]:
# Phishing-only datasets shouldn't have spam=1
bad_spam = combined[
    combined["source_dataset"].isin(["Nazario_5", "Nigerian_5"])
    & (combined["label_3class"] == 1)
]

# Spam-only datasets shouldn't have phishing=2
bad_phish = combined[
    combined["source_dataset"].isin(["SpamAssasin", "TREC"])
    & (combined["label_3class"] == 2)
]

print("Nazario/Nigerian with spam label:", len(bad_spam))
print("SpamAssassin/TREC with phishing label:", len(bad_phish))


Nazario/Nigerian with spam label: 0
SpamAssassin/TREC with phishing label: 0


**Check for duplicate values**: Making sure that within the dataset, there are no two instances that have the exact same values for all attributes.

In [ ]:
dedup_subset = ["source_dataset", "sender", "receiver", "date", "subject", "body"]

dupes = combined[combined.duplicated(subset=dedup_subset, keep=False)]
print("Total duplicate rows (by subset):", len(dupes))

dupes[["source_dataset", "sender", "date", "subject"]].head(10)


Total duplicate rows (by subset): 0


,source_dataset,sender,date,subject




**Checking is structure/format is correct:**

In [ ]:
#Checking to see if URLs are list
is_list = combined["urls"].apply(lambda x: isinstance(x, list))
print("Non-list urls:", (~is_list).sum())

combined.loc[~is_list, ["source_dataset", "urls"]].head(10)


Non-list urls: 0


,source_dataset,urls


In [ ]:
#Missing sender / reciever / emails
print("Missing sender:", combined["sender"].isna().sum())
print("Missing receiver:", combined["receiver"].isna().sum())

print("Missing sender_email:", combined["sender_email"].isna().sum())
print("Missing sender_domain:", combined["sender_domain"].isna().sum())


Missing sender: 0
Missing receiver: 1476
Missing sender_email: 0
Missing sender_domain: 0


In [ ]:
no_text = combined[combined["subject"].isna() & combined["body"].isna()]
print("Rows with no subject and no body:", len(no_text))


Rows with no subject and no body: 0


**Checking for wrong dates:**

In [ ]:
bad_dates = combined[(combined["year"] < 1999) | (combined["year"] > 2025)]
bad_dates[["source_dataset", "date", "subject"]].head(20)


,source_dataset,date,subject


<h2 style="border-bottom:3px solid #f59e0b; padding-bottom:4px;">
🧬 Step 10 – Near-duplicate detection using TF-IDF + k-NN
</h2>

<p>
To understand redundancy in the corpus, we use TF-IDF features plus cosine
nearest neighbors to detect <strong>near-duplicate emails</strong>.
We run this both on:
</p>

<ul>
  <li><strong>Subject + body combined</strong>: captures near-identical full messages.</li>
  <li><strong>Body only</strong>: focuses on content similarities even when subjects differ.</li>
</ul>

<p>
The counts of highly similar pairs (e.g., cosine similarity ≥ 0.99) help quantify how often
templates or repeated campaigns appear across the datasets.
</p>


**Checking for near duplicates:**

Subject + body text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

# 1. Combine subject + body into one text field
combined["subject_body"] = (
    combined["subject"].fillna("") + " " + combined["body"].fillna("")
)

# 2. TF-IDF (simple but works well for near-dup detection)
tfidf = TfidfVectorizer(
    lowercase=True,
    max_df=0.8,        # ignore extremely common terms
    min_df=5,          # ignore super-rare terms
    ngram_range=(1, 2) # unigrams + bigrams
)

X = tfidf.fit_transform(combined["subject_body"])
print("TF-IDF shape:", X.shape)


TF-IDF shape: (17407, 152756)


In [ ]:
# 3. Nearest neighbor search in TF-IDF space
nn = NearestNeighbors(
    metric="cosine",
    algorithm="brute",
    n_neighbors=5,   # top 5 neighbors per email
    n_jobs=-1,
)
nn.fit(X)

distances, indices = nn.kneighbors(X)
similarities = 1 - distances  # cosine similarity = 1 - distance


In [ ]:
pairs = []
threshold = 0.99

n_rows = X.shape[0]
for i in range(n_rows):
    # indices[i, 0] is i itself, so start from 1
    for j_idx, sim in zip(indices[i, 1:], similarities[i, 1:]):
        if sim >= threshold:
            a, b = sorted((i, j_idx))
            pairs.append((a, b, sim))

pairs_df = (
    pd.DataFrame(pairs, columns=["idx1", "idx2", "similarity"])
    .drop_duplicates(subset=["idx1", "idx2"])
    .sort_values("similarity", ascending=False)
)

print("Number of near-duplicate pairs (sim >= {:.2f}):".format(threshold), len(pairs_df))
pairs_df.head(10)


Number of near-duplicate pairs (sim >= 0.99): 2335


,idx1,idx2,similarity
3026,17384,17384,1.0
0,5,5,1.0
1,5,6,1.0
3025,17381,17383,1.0
3024,17383,17384,1.0
4,85,380,1.0
3023,17381,17381,1.0
3022,17381,17384,1.0
3021,17377,17378,1.0
3020,17377,17377,1.0


**Checking for near duplicates:**

Body text only

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

# 1. Use ONLY the body text
combined["body_only"] = combined["body"].fillna("").astype(str)

tfidf_body = TfidfVectorizer(
    lowercase=True,
    max_df=0.8,        # ignore extremely common terms
    min_df=5,          # ignore super-rare terms
    ngram_range=(1, 2) # unigrams + bigrams
)

X_body = tfidf_body.fit_transform(combined["body_only"])
print("TF-IDF (body-only) shape:", X_body.shape)


TF-IDF (body-only) shape: (17407, 150041)


In [ ]:
nn_body = NearestNeighbors(
    metric="cosine",
    algorithm="brute",
    n_neighbors=5,
    n_jobs=-1,
)
nn_body.fit(X_body)

distances_body, indices_body = nn_body.kneighbors(X_body)
similarities_body = 1 - distances_body


In [ ]:
pairs_body = []
threshold = 0.99
n_rows = X_body.shape[0]

for i in range(n_rows):
    for j_idx, sim in zip(indices_body[i, 1:], similarities_body[i, 1:]):
        if j_idx == i:
            continue          # skip self
        if sim >= threshold:
            a, b = sorted((i, j_idx))
            pairs_body.append((a, b, sim))

pairs_body_df = (
    pd.DataFrame(pairs_body, columns=["idx1", "idx2", "similarity"])
    .drop_duplicates(subset=["idx1", "idx2"])
    .sort_values("similarity", ascending=False)
)

print("Number of near-duplicate BODY pairs (sim >= {:.2f}):"
      .format(threshold), len(pairs_body_df))
pairs_body_df.head(10)


Number of near-duplicate BODY pairs (sim >= 0.99): 2121


,idx1,idx2,similarity
12,115,4675,1.0
2978,17396,17402,1.0
2977,430,17387,1.0
10,80,2717,1.0
8,52,7639,1.0
30,156,3648,1.0
28,154,14503,1.0
27,154,3927,1.0
26,145,146,1.0
2900,17136,17138,1.0


<h2 style="border-bottom:3px solid #06b6d4; padding-bottom:4px;">
🔗 Step 11 – URL coverage analysis
</h2>

<p>
Since later models may use <strong>URL domains</strong> as features (and for RDAP/DNS lookup),
we measure how many emails actually contain at least one URL.
This gives a quick sense of how strong URL-based features can be in the combined corpus.
</p>

<div style="padding:10px; background:#ecfeff; border-left:5px solid #0891b2; margin-top:6px;">
The resulting percentages guide decisions like:  
“Should URL-focused models be primary signals, or auxiliary signals?”
</div>


In [ ]:
# Count emails with at least one URL
mask_has_url = combined["urls"].apply(lambda x: isinstance(x, list) and len(x) > 0)

num_with_urls = mask_has_url.sum()
num_total = len(combined)
num_without_urls = num_total - num_with_urls

print("Total emails:", num_total)
print("Emails WITH URLs:", num_with_urls)
print("Emails WITHOUT URLs:", num_without_urls)
print("Percentage with URLs: {:.2f}%".format(100 * num_with_urls / num_total))


Total emails: 17407
Emails WITH URLs: 810
Emails WITHOUT URLs: 16597
Percentage with URLs: 4.65%


### Data Pre-processing


**Objective**: Clean all noisy data within both datasets.

**Task**: Use various data preprocessing strategies to remove all problems within the dataset that might hinder various algorithms we may use in data classification.

These are just few things that come into my mind on how I would start preprocessing the data...

- **Data Cleaning**
    - First I plan to remove all empty, NULL, or irrelevant records from the dataset. Irrelevant records include all that have a timestamp greater than 5 years or no timestamp at all. This is to ensure that the dataset is at least somewhat relevant to how phishing email attacks are at this point in time.
    - For the missing values within this dataset, depending on what exact feature is absent I'll either just fill with a placeholder or exempt it from my model. For example...
        - If either the subject, body, or URL is empty I'll just fill a placeholder
        - However, if either the sender or reciever is empty I'll exempt it from use due to my emphasis on using the domains.
    - Then I'll transform the text into an easy to understand format for machine learning by lowercasing all text, removing extra spaces, and any punctuation and emojis.
    - The Nazario5 doesn't have any duplicates, but if it did I would just ignore them.
- **Tokenization**
    - I'll use a *subword tokenizer* to map each word or part-word into a integer-based ID using a pretrained vocabulary.
- **Stopword Removal**
    - To reduce noise within the data I'll remove common words and phrase that are popular in English vocabulary but not important to the data at all.
        - For example, "the", "and", "your", etc.
- **Word Embeddings**
    - Then once we have created a token system that maximizes capturing the relevant info needed we will turn these tokens into numerical vectors (word embeddings). All word_embeddings will also be mapped with a positional_embedding which will result in the embedding matrix we will use for actual input.
    - Freeze embeddings?

One guarantee action of Data Transformation I will do, is creating new features to represent RDAP and DNS information obtained from domains

### Analyzing the datasets:

### Classification (Model Training)


**Objective**: Classify whether an email is a phishing attack or not.

**Task**: First, we will implement 2 basic models that work based on word embeddings for a baseline on testing how well classify whether an email is a phishing attack or not through purely using subject/body text.

Afterwards, we'll evaluate performance on how well they classified the original email dataset and then how well the performed under TextAttack.

After training and evaluating the baseline models, we'll move on to models that use Tabular embeddings for the RDAP/DNS lookup features. We'll prepare 2 types of those models and pair each with a baseline model to implement a 2-stage pipeline classifier.

Afterwards, we'll evaluate both piplines' performance and robustness under TextAttack.

Finally, based on the evaluation results we'll convert the best performing pipeline to process the data in real-time. Then, we'll compare performance of the models when they were doing batch processing vs real-time processing.
- It'll be important to note that after we deploy the models in real-time, we'll have to evalute not only the difference in how well they classify the emails but also how well the model runs. As we also assume that TextAttack might try to DDoS our model and thus we'll have to also implement defensive stratgies for that.

Then, after we either successfully defend our model in real-time or report that we failed to do so. We'll gather all results in an easy manner to see and evaluate the differences.

**Baseline Pipeline:**
- TF-IDF + Logistic Regression for the subject/body text.
- AdaBoost for the RDAP/DNS lookups.
- Evaluate performance (& robustness?) for both models individually.
- Then fuse Logistic Regression on [text_prob, tabs_prob] to create a 2-stage pipeline.
- Evaluate pipeline performance/robustness under TextAttack.


**Deep Pipeline:**
- I'll either use a One-Class SVM or a Isolation Forest as a preliminarily model to tag "suspicious" emails. (Anomaly gate)
- Then I'll use DistilBert for the subject/body text.
- And LightGBM for the RDAP/DNS lookups.
- First, I'll evaluate each of these models individual as their own classifier. Evaluating their performance/robustness.
- Then, I'll use the 3 models to build a 3-stage pipeline, then evaluate the pipelines' performance against the baseline.

After comparing results of each pipeline, *if time allows me* I'll also deploy both models in real-time then compare not only performance or robustness, but also further metrics like latency, availability, reliability, & operational costs.

#### Unstructured Learning (Subject/body text)

**Term Frequency Inverse Document Frequency (TF-IDF):** TF-IDF is a statistical measure that reflects how important a word is to a single email within a collection of emails. Like the name, it utilizes 2 functions:
$$Term~~Frequency = \frac{\#~occurences~~of~~'t'~~within~~the~~entire~~dataset}{\#~total~~words~~within~~the~~entire~~dataset}$$

where *t* represents a given word.

$$idf(t, D) = \log{\frac{N}{\#~of~~emails~~that~~contain~~'t'}}$$

where *t* represents a given word, and D represents the entire corpus of emails (the dataset).

Through these, we are able to assign a numerical value to a word that represents how often and how rare that certain word is in regards to the entire collection of emails we have within the dataset.

**+ Logistic Regression:** Then once we have turned the text into numberrs through TF-IDF, we would use a simple Logistic Regression model like...

$$ P(phishing) = \frac{1}{1 + e^{-(w_0+w_1+w_2...w_n)}}$$

which will estimate the probability of a email being a phishing attack where each $w_i$ is a weight learned during training.

##### **Encoder-Only Transformer: (BERT)**
A type of model that takes text as input and outputs contextualized embeddings — meaning it represents what each word means given the words around it.


Similar to a TextCNN, a encoder-only transformer does require inputs as word embeddings; however, it requires a specific "type" of embedding through <u>*subword tokenization*</u> where text is split into smaller pieces called *tokens*, for example:
   - Whole words like, "bank"
   - Subwords like ("ver", "ify", "account")
   - Punctuation ("." or "," or "##ing")

These consists of the actual *token embedding* which captures the semantic meaning of the word or text. However, transformers a few more embeddings to get a deeper understanding of the text such as...
   - Positional embedding, which represents word order for the model.
   - Segment embedding, which helps the model relate which sentence relates to which token.

After the token embeddings are created, we feed them to two important sub-layers:
1. **Multi-Head Self-Attention**
2. **Feed-Forward Network (FFN)**

**Self-Attention:** This layer represents how the models learns how much *"attention"* it should pay to every other word. It computes this through 3 vectors for each token:
   - Q (Query) - What token I'm looking for
   - K (Key) - Token identifcation?
   - V (Value) - What info the token offers

And through these, it calculates attention weights as:
$$Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})$$

With this, each word's representation becomes a weight blend that relates to all other words vlaues.


**Multi-Head Mechanism:** With this mechanism, the model runs multiple of these attention functions in parallel (i.e, running multiple *"heads"*). Through these heads each learn a different relationship, such as:
   - Head 1: subject-verb relation
   - Head 2: urgency words
   - Head 3: negations, etc.

Then the outputs from all heads are put together and projected into one representation through the Feed-Forward Network.


**Feed-Forward Network:** After attention, each token embedding passes through a small fully connected network:
$$FFN(x) =  ReLU(xW_1 + b_1 ) W_2 + b_2 $$
where $x$ represents...


**Residual + Layer Norm:** To stabilize traiing, each sub-layer adds the original input back (through residual connection) and applies layer normalization:
$$ x' = LayerNorm(x + Sublayer(x)) $$


**Stacking Layers:** And all of this forms a single encoder block for our models. Within a encoder-only transformer,  multiple encoder blocks are stacked on top of each other where each layer refines the contextual meaning of every token:
   - Layer 1: word-level context
   - Layer 4: phrase-level
   - Layer 12: sentence-level semantics

And thus after the last encoder layer, you typically take the embedding corresponding to the special CLS token (which represents the whole sentence) and put that through a *classification head* (a small dense + softmax/sigmoid layer) to get:
$$ P(phishing) = \sigma(W \times h_{[CLS]} + b)$$

<h2 style="border-bottom:3px solid #22c55e; padding-bottom:4px;">
🕰️ Step 12 – Create balanced pre-2010 and post-2010 subsets
</h2>

<p>
To study temporal drift and model behavior across eras, we split the corpus into:
</p>

<ul>
  <li><strong>Pre-2010 emails</strong></li>
  <li><strong>2010+ emails</strong></li>
</ul>

<p>
We then downsample the larger side so both groups have the same number of emails
(<code>target_size</code>), and save:
</p>

<ul>
  <li><code>emails_before2010_balanced.csv</code></li>
  <li><code>emails_after2010_balanced.csv</code></li>
</ul>

<div style="padding:10px; background:#f0fdf4; border-left:5px solid #16a34a; margin-top:6px;">
These balanced splits are used for <strong>pre- vs post-2010 embedding and robustness comparisons</strong>.
</div>


In [ ]:
# ============================================================
# Step 12 – Create balanced pre-2010 and post-2010 subsets
#   A) Same as before (2000 per era, real label ratios)
#   B) NEW: label-balanced post-2010 split (much more ham share)
# ============================================================

# ---- A) ORIGINAL YEAR-BASED BALANCED SPLITS (unchanged) ----
year_counts = combined["year"].value_counts().sort_index()
print("Email counts per year:")
print(year_counts)

TARGET_SIZE = 2000  # per era

before_2010 = combined[combined["year"] < 2010].copy()
after_2010  = combined[combined["year"] >= 2010].copy()

print("\nTotal emails before 2010:", len(before_2010))
print("Total emails from 2010 onward:", len(after_2010))

replace_before = len(before_2010) < TARGET_SIZE
replace_after  = len(after_2010)  < TARGET_SIZE

before_balanced = before_2010.sample(
    n=TARGET_SIZE,
    random_state=SEED,
    replace=replace_before,
)
after_balanced = after_2010.sample(
    n=TARGET_SIZE,
    random_state=SEED,
    replace=replace_after,
)

print("\nBalanced subset sizes (year-based):")
print("Before 2010:", len(before_balanced))
print("After 2010 :", len(after_balanced))

out_before = os.path.join(PATH, "emails_before2010_balanced.csv")
out_after  = os.path.join(PATH, "emails_after2010_balanced.csv")

before_balanced.to_csv(out_before, index=False)
after_balanced.to_csv(out_after,  index=False)

print("\nSaved year-balanced subsets to:")
print(" ", out_before)
print(" ", out_after)

print("\nLabel distribution (3-class) in year-balanced BEFORE 2010:")
print(before_balanced["label_3class"].value_counts().sort_index())

print("\nLabel distribution (3-class) in year-balanced AFTER 2010:")
print(after_balanced["label_3class"].value_counts().sort_index())

# ---- B) NEW: LABEL-BALANCED POST-2010 SPLIT ----------------
print("\nPost-2010 label counts in FULL data:")
print(after_2010["label_3class"].value_counts().sort_index())

# We can only balance up to the minority class (ham)
per_class_target = after_2010["label_3class"].value_counts().min()
print(f"\nBuilding label-balanced post-2010 subset with {per_class_target} per class.")

def sample_per_label(df, label, n, seed):
    df_lab = df[df["label_3class"] == label]
    return df_lab.sample(
        n=n,
        random_state=seed,
        replace=(len(df_lab) < n)
    )

balanced_chunks = []
for lab in [0, 1, 2]:  # 0=ham, 1=spam, 2=phishing
    chunk = sample_per_label(after_2010, lab, per_class_target, SEED)
    balanced_chunks.append(chunk)

after_label_balanced = pd.concat(balanced_chunks, ignore_index=True)
after_label_balanced = after_label_balanced.sample(
    frac=1.0, random_state=SEED
).reset_index(drop=True)

print("\nLabel distribution in label-balanced AFTER-2010 subset:")
print(after_label_balanced["label_3class"].value_counts().sort_index())
print("Total rows in label-balanced AFTER-2010 subset:", len(after_label_balanced))

out_after_label = os.path.join(PATH, "emails_after2010_labelbalanced.csv")
after_label_balanced.to_csv(out_after_label, index=False)
print("\nSaved label-balanced post-2010 subset to:")
print(" ", out_after_label)


Email counts per year:
year
2000.0      13
2001.0     776
2002.0    4605
2003.0     374
2004.0     408
2005.0     528
2006.0     986
2007.0    6248
2008.0    1468
2009.0       1
2015.0      63
2016.0     295
2017.0     194
2018.0     166
2019.0      85
2020.0      53
2021.0      29
2022.0      67
2023.0     801
2024.0      20
2025.0     227
Name: count, dtype: int64

Total emails before 2010: 15407
Total emails from 2010 onward: 2000

Balanced subset sizes (year-based):
Before 2010: 2000
After 2010 : 2000

Saved year-balanced subsets to:
  /content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/emails_before2010_balanced.csv
  /content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/emails_after2010_balanced.csv

Label distribution (3-class) in year-balanced BEFORE 2010:
label_3class
0    1103
1     571
2     326
Name: count, dtype: Int64

Label distribution (3-cla

In [ ]:
# Count emails per year
year_counts = combined['year'].value_counts().sort_index()

print(year_counts)


year
2000.0      13
2001.0     776
2002.0    4605
2003.0     374
2004.0     408
2005.0     528
2006.0     986
2007.0    6248
2008.0    1468
2009.0       1
2015.0      63
2016.0     295
2017.0     194
2018.0     166
2019.0      85
2020.0      53
2021.0      29
2022.0      67
2023.0     801
2024.0      20
2025.0     227
Name: count, dtype: int64


In [ ]:
before_2010 = combined[combined['year'] < 2010]
print("Emails before 2010:", len(before_2010))

after_2010 = combined[combined['year'] >= 2010]
print("Emails from 2010 and after:", len(after_2010))

Emails before 2010: 15407
Emails from 2010 and after: 2000


In [ ]:

before_2010 = combined[combined['year'] < 2010]
after_2010 = combined[combined['year'] >= 2010]

print(len(before_2010), len(after_2010))


15407 2000


In [ ]:
target_size = min(len(before_2010), len(after_2010))
print("Target size per group:", target_size)


Target size per group: 2000


In [ ]:
before_balanced = before_2010.sample(n=target_size, random_state=SEED)
after_balanced  = after_2010.sample(n=target_size, random_state=SEED)


In [ ]:
before_balanced.isna().sum()

,0
sender,0
receiver,178
date,0
subject,0
body,0
label,0
urls,0
sender_emails,0
receiver_emails,0
sender_email,0


In [ ]:
after_balanced.isna().sum()

,0
sender,0
receiver,76
date,0
subject,0
body,0
label,0
urls,0
sender_emails,0
receiver_emails,0
sender_email,0


In [ ]:
before_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 11499 to 13285
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   sender                  2000 non-null   string        
 1   receiver                1822 non-null   string        
 2   date                    2000 non-null   datetime64[ns]
 3   subject                 2000 non-null   string        
 4   body                    2000 non-null   string        
 5   label                   2000 non-null   object        
 6   urls                    2000 non-null   object        
 7   sender_emails           2000 non-null   object        
 8   receiver_emails         2000 non-null   object        
 9   sender_email            2000 non-null   object        
 10  sender_domain           2000 non-null   object        
 11  url_domains             2000 non-null   object        
 12  source_dataset          2000 non-null   object  

In [ ]:
after_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 16875 to 7593
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   sender                  2000 non-null   string        
 1   receiver                1924 non-null   string        
 2   date                    2000 non-null   datetime64[ns]
 3   subject                 2000 non-null   string        
 4   body                    2000 non-null   string        
 5   label                   2000 non-null   object        
 6   urls                    2000 non-null   object        
 7   sender_emails           2000 non-null   object        
 8   receiver_emails         2000 non-null   object        
 9   sender_email            2000 non-null   object        
 10  sender_domain           2000 non-null   object        
 11  url_domains             2000 non-null   object        
 12  source_dataset          2000 non-null   object   

In [ ]:
before_balanced_path = os.path.join(PATH, "emails_before2010_balanced.csv")
before_balanced.to_csv(before_balanced_path, index=False)
print("Saved:", before_balanced_path)


Saved: /content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/emails_before2010_balanced.csv


In [ ]:
after_balanced_path = os.path.join(PATH, "emails_after2010_balanced.csv")
after_balanced.to_csv(after_balanced_path, index=False)
print("Saved:", after_balanced_path)


Saved: /content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/emails_after2010_balanced.csv


<h2 style="border-bottom:3px solid #8b5cf6; padding-bottom:4px;">
🤖 Step 13 – Build a synthetic email dataset from LLM outputs
</h2>

<p>
In addition to authentic emails, we construct a <strong>synthetic dataset</strong> composed of
LLM-generated messages from multiple models (OpenAI, DeepSeek, etc.).
We:
</p>

<ol>
  <li>Filter <code>merged_all_emails.csv</code> to keep only <strong>non-human</strong> rows.</li>
  <li>Map textual <code>Type</code> labels into the same 3-class schema.</li>
  <li>Normalize into the common email schema using <code>normalize_email_frame</code>.</li>
  <li>Use TF-IDF + k-NN + union-find to cluster near-duplicate synthetic emails.</li>
  <li>Sample a balanced, de-duplicated set of <strong>2000 synthetic emails</strong>
      with a 45/35/20 ham/spam/phishing ratio.</li>
  <li>Save the final synthetic corpus as <code>emails_synthetic.csv</code>.</li>
</ol>

<div style="padding:10px; background:#f5f3ff; border-left:5px solid #7c3aed; margin-top:6px;">
This synthetic dataset lets us study <strong>distribution shift</strong> and robustness when models are
trained on real emails but evaluated on high-quality LLM-generated phishing content.
</div>


In [ ]:
df = pd.read_csv(PATH + "merged_all_emails.csv")

df["Created by"].dropna().unique()


array(['Human', 'Huamn', 'deepseek-chat', 'OpenAI',
       'LLM (Amazon Nova Lite v1)', 'LLM (Gemini 1.5 Pro)',
       'LLM (Mistral Medium 3.1)', 'LLM (Grok 4)', 'LLM (GPT-4o)',
       'LLM (GPT-5 Mini)', 'LLM (Amazon Nova Pro v1)',
       'LLM (DeepSeek Chat)', 'LLM (Qwen2.5 VL 32B Free)',
       'LLM (Codestral 2508)', 'LLM (Qwen2.5 Coder 7B)',
       'LLM (Llama 3.3 8B Instruct Free)', 'openai/gpt-5-mini',
       'mistralai/codestral-2508', 'LLM (Llama Guard 3 8B)',
       'google/gemini-1.5-pro', 'openai/gpt-4o',
       'meta-llama/llama-3.3-8b-instruct:free', 'deepseek/deepseek-chat',
       'qwen/qwen2.5-vl-32b-instruct:free', 'meta-llama/llama-guard-3-8b',
       'amazon/nova-pro-v1', 'x-ai/grok-4', 'amazon/nova-lite-v1',
       'qwen/qwen2.5-coder-7b-instruct', 'mistralai/mistral-medium-3.1'],
      dtype=object)

In [ ]:
df.head()

,No.,Original email No.,Subject,Body,Sender,URL(s),File,LLM detected emotion,LLM detected motivation,Type,Created by,Source,Year
0,1,1,New login on your pCloud account,Did you recently log in? Someone just tried to...,pCloud Team <noreply@isaccess.co.za>,NaN,NaN,"['Fear', 'Urgency', 'Authority']",['Follow the link'],Phishing,Human,Manual,2025
1,2,2,INVITATION TO THE 79TH IFALPA ANNUAL CONFERENCE,"Dear Anonymous, On behalf of the IFALPA local ...",International Federation Of Air Line Pilots A...,NaN,NaN,"['Urgency', 'Authority', 'Greed']",['Reply'],Phishing,Human,Manual,2025
2,3,3,bitcoin payment,Hello Your Wallet is completed and has been fu...,PERROT Hugo <Hugo.PERROT@SDIS78.FR>,NaN,NaN,"['Greed', 'Curiosity']",['Follow the link'],Phishing,Human,Manual,2025
3,4,4,DHL Megjegyzés: A csomag kiadásához vámkezelé...,Tisztelt DHL Ügyfél! A DHL csomagja kézbesítés...,DHL Package <dhl@shop.dunacademy.info>,https://tinyurl.com/3e6e4yuw,NaN,"['Urgency', 'Fear', 'Authority']",['Follow the link'],Phishing,Human,Manual,2025
4,5,5,Quote Request - Mol Group,"Greetings, Kindly source for the below items a...",Mol Group <supplies@molgr-team.info>,NaN,NaN,"['Urgency', 'Authority']",['Reply'],Phishing,Human,Manual,2025


What kind of synthetic data are we using?

In [ ]:
human_values = ["human", "huamn"]
mask_nonhuman = ~df["Created by"].str.lower().isin(human_values)

df_nonhuman = df[mask_nonhuman]


In [ ]:
creator_counts = (
    df_nonhuman["Created by"]
    .value_counts()
    .sort_values(ascending=False)
)

print(creator_counts)


Created by
deepseek-chat                            3075
OpenAI                                   3075
LLM (GPT-4o)                              340
LLM (DeepSeek Chat)                       336
LLM (Mistral Medium 3.1)                  326
LLM (Qwen2.5 Coder 7B)                    319
LLM (Codestral 2508)                      313
LLM (Grok 4)                              309
LLM (Amazon Nova Pro v1)                  304
LLM (Qwen2.5 VL 32B Free)                 298
LLM (Amazon Nova Lite v1)                 280
LLM (Llama 3.3 8B Instruct Free)           60
LLM (Gemini 1.5 Pro)                       59
LLM (GPT-5 Mini)                           55
LLM (Llama Guard 3 8B)                     51
x-ai/grok-4                                 4
google/gemini-1.5-pro                       3
mistralai/codestral-2508                    2
qwen/qwen2.5-vl-32b-instruct:free           2
openai/gpt-5-mini                           2
qwen/qwen2.5-coder-7b-instruct              2
mistralai/mistral-mediu

In [ ]:
top10 = creator_counts.head(10)

top10_total = top10.sum()
print("Total emails from top 10 synthetic creators:", top10_total)


Total emails from top 10 synthetic creators: 8695


In [ ]:
top10_creators = creator_counts.head(10).index.tolist()


df_top10 = df_nonhuman[df_nonhuman["Created by"].isin(top10_creators)]

distribution = (
    df_top10
    .groupby(["Created by", "Type"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)

print(distribution)


Type                       Phishing  Phishing Simulation  Spam  Valid
Created by                                                           
LLM (Amazon Nova Pro v1)        119                    8    89     88
LLM (Codestral 2508)            119                    5   102     87
LLM (DeepSeek Chat)             110                    2   111    113
LLM (GPT-4o)                    106                    0   126    108
LLM (Grok 4)                    103                    5   100    101
LLM (Mistral Medium 3.1)         94                    7   112    113
LLM (Qwen2.5 Coder 7B)          119                    1   106     93
LLM (Qwen2.5 VL 32B Free)       138                    5    81     74
OpenAI                         1090                   35  1000    950
deepseek-chat                  1090                   35  1000    950


<h2 style="border-bottom:3px solid #ef4444; padding-bottom:4px;">
🧪 Step 14 – Validate diversity of the final synthetic dataset
</h2>

<p>
As a final sanity check, we re-run TF-IDF + cosine nearest neighbors on the
<strong>final selected synthetic emails</strong> to make sure we did not accidentally keep
large clusters of duplicates.
</p>

<p>
Very few near-duplicate pairs at high similarity thresholds confirm that the
synthetic corpus is <strong>diverse enough</strong> for meaningful evaluation and embedding analysis.
</p>


In [ ]:

# 1) Load merged_all_emails and keep ONLY non-human rows
df_all = pd.read_csv(os.path.join(PATH, "merged_all_emails.csv"))

human_values = ["human", "huamn"]
mask_nonhuman = ~df_all["Created by"].str.lower().isin(human_values)
df_synth_raw = df_all[mask_nonhuman].copy()

print("Total rows in merged_all_emails:", len(df_all))
print("Non-human rows (synthetic pool):", len(df_synth_raw))

# Drop exact duplicates based on raw text + Type
dupe_subset = ["Subject", "Body", "Type"]
before = len(df_synth_raw)
df_synth_raw = df_synth_raw.drop_duplicates(subset=dupe_subset, keep="first")
after = len(df_synth_raw)
print("Rows after dropping exact Subject+Body+Type duplicates:", after)
print("Exact duplicates removed:", before - after)

# Map Type -> 3-class labels
def map_type_to_3class_local(val):
    if pd.isna(val):
        raise ValueError("Synthetic row has missing Type/label")
    s = str(val).strip().lower()
    if "valid" in s or "legit" in s or "ham" in s:
        return 0       # ham_legit
    if "phishing" in s:
        return 2       # phishing (incl. simulations)
    if "spam" in s or "malicious" in s:
        return 1       # spam
    raise ValueError(f"Unrecognized Type label: {val!r}")

df_synth_raw["label_3class"] = (
    df_synth_raw["Type"].apply(map_type_to_3class_local).astype("Int64")
)

print("Pool label_3class counts:")
print(df_synth_raw["label_3class"].value_counts().sort_index())


Total rows in merged_all_emails: 12300
Non-human rows (synthetic pool): 9226
Rows after dropping exact Subject+Body+Type duplicates: 8716
Exact duplicates removed: 510
Pool label_3class counts:
label_3class
0    2660
1    2704
2    3352
Name: count, dtype: Int64


In [ ]:
# 2) Build minimal schema and normalize entire synthetic pool

df_pool_min = df_synth_raw[
    ["Subject", "Body", "Sender", "URL(s)", "Type", "Year", "label_3class"]
].copy()

df_pool_min = df_pool_min.rename(columns={
    "Subject": "subject",
    "Body": "body",
    "Sender": "sender",
    "URL(s)": "urls",
    "Type": "label",
})

df_pool_min["receiver"] = "[UNKNOWN]"

rng = np.random.default_rng(SEED)
df_pool_min["date"] = (
    pd.to_datetime(df_pool_min["Year"].astype(int).astype(str), format="%Y") +
    pd.to_timedelta(rng.integers(0, 365, size=len(df_pool_min)), unit="D")
)

df_pool_min = df_pool_min[
    ["sender", "receiver", "date", "subject", "body", "label", "urls", "label_3class"]
]

df_pool_norm = normalize_email_frame(
    df_pool_min, source_name="MergedAll_SynthPool"
)

# Carry label_3class through explicitly
df_pool_norm["label_3class"] = df_pool_min["label_3class"].values

print("Normalized synthetic pool shape:", df_pool_norm.shape)
df_pool_norm.head(3)

Normalized synthetic pool shape: (8716, 16)


,sender,receiver,date,subject,body,label,urls,label_3class,sender_emails,receiver_emails,sender_email,receiver_email_primary,sender_domain,receiver_domains,url_domains,source_dataset
52,[CFP] ICEEECS2024<iceeecs@setteknoloji.news-ma...,[UNKNOWN],2025-10-13,[CFP] ICEEECS2024 - Ankara - Hybrid,"Please mark this email as ""NOT SPAM"" in order ...",Spam,[],1,[iceeecs@setteknoloji.news-mailer.com],[],iceeecs@setteknoloji.news-mailer.com,<NA>,setteknoloji.news-mailer.com,[],[],MergedAll_SynthPool
3075,pCloud Team <noreply@isaccess.co.za>,[UNKNOWN],2025-06-01,Unfamiliar login attempt detected on your pClo...,Was this your recent login? We've detected a l...,Phishing,[],2,[noreply@isaccess.co.za],[],noreply@isaccess.co.za,<NA>,isaccess.co.za,[],[],MergedAll_SynthPool
3076,International Federation Of Air Line Pilots A...,[UNKNOWN],2025-07-24,OFFICIAL INVITATION: 79TH IFALPA ANNUAL CONFER...,"Dear Anonymous, The IFALPA local organizing co...",Phishing,[],2,[IFALPA2025CONFERENCE@execs.com],[],ifalpa2025conference@execs.com,<NA>,execs.com,[],[],MergedAll_SynthPool


In [ ]:
# 3) Filter to rows with valid sender_domain and set helper label fields

before = len(df_pool_norm)
df_pool_norm = df_pool_norm[df_pool_norm["sender_domain"].notna()].copy()
print("Dropped rows with missing sender_domain:", before - len(df_pool_norm))
print("Rows left after domain filter:", len(df_pool_norm))

# Binary label + name
df_pool_norm["label_binary"] = (df_pool_norm["label_3class"] == 2).astype("int64")
label_names = {0: "ham_legit", 1: "spam", 2: "phishing"}
df_pool_norm["label_name"] = df_pool_norm["label_3class"].map(label_names)

df_pool_norm["year"] = df_pool_norm["date"].dt.year
df_pool_norm["subject_len"] = df_pool_norm["subject"].fillna("").str.len()
df_pool_norm["body_len"] = df_pool_norm["body"].fillna("").str.len()

print("Post-filter label_3class counts:")
print(df_pool_norm["label_3class"].value_counts().sort_index())


Dropped rows with missing sender_domain: 201
Rows left after domain filter: 8515
Post-filter label_3class counts:
label_3class
0    2654
1    2701
2    3160
Name: count, dtype: Int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# 4) TF-IDF on full synthetic pool (body-only) + near-duplicate clusters

body_text_pool = df_pool_norm["body"].fillna("").astype(str)

tfidf_body_pool = TfidfVectorizer(
    lowercase=True,
    max_df=0.8,
    min_df=5,
    ngram_range=(1, 2),
    stop_words="english",
    max_features=5000,
)
X_body_pool = tfidf_body_pool.fit_transform(body_text_pool)

print("TF-IDF (pool) shape:", X_body_pool.shape)

nn_pool = NearestNeighbors(
    metric="cosine",
    algorithm="brute",
    n_neighbors=5,
    n_jobs=-1,
)
nn_pool.fit(X_body_pool)

distances_pool, indices_pool = nn_pool.kneighbors(X_body_pool)
similarities_pool = 1 - distances_pool

threshold = 0.95
pairs = []
n_rows = X_body_pool.shape[0]

for i in range(n_rows):
    for j_idx, sim in zip(indices_pool[i, 1:], similarities_pool[i, 1:]):
        if j_idx == i:
            continue
        if sim >= threshold:
            a, b = sorted((i, j_idx))
            pairs.append((a, b))

pairs = list(set(pairs))  # unique (a,b)
print("Near-duplicate BODY pairs in pool (sim >= 0.95):", len(pairs))


TF-IDF (pool) shape: (8515, 5000)
Near-duplicate BODY pairs in pool (sim >= 0.95): 2882


In [ ]:
# 5) Union-Find clustering on near-duplicate pairs

parent = list(range(n_rows))

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for a, b in pairs:
    union(a, b)

cluster_id = [find(i) for i in range(n_rows)]
df_pool_norm["cluster_id"] = cluster_id

n_clusters = len(set(cluster_id))
print("Total near-dup clusters in pool:", n_clusters)


Total near-dup clusters in pool: 6832


In [ ]:
# 6) Sample 2002 rows with 45/35/20 split, 1 per near-dup cluster
N_TOTAL = 2000
TARGET_PROPS = {0: 0.45, 1: 0.35, 2: 0.20}
target_counts = {lab: int(round(p * N_TOTAL)) for lab, p in TARGET_PROPS.items()}

diff = N_TOTAL - sum(target_counts.values())
if diff != 0:
    largest = max(target_counts, key=target_counts.get)
    target_counts[largest] += diff

print("Target counts:", target_counts)  # should show {0: 901, 1: 701, 2: 400}

rng = np.random.RandomState(SEED)
used_clusters = set()
selected_rows = []

for lab in [0, 1, 2]:
    df_lab = df_pool_norm[df_pool_norm["label_3class"] == lab]
    groups = df_lab.groupby("cluster_id")
    cluster_ids = list(groups.groups.keys())

    avail = [cid for cid in cluster_ids if cid not in used_clusters]
    if len(avail) < target_counts[lab]:
        raise ValueError(
            f"Not enough distinct clusters for label {lab}: "
            f"need {target_counts[lab]}, have {len(avail)}"
        )

    chosen = rng.choice(avail, size=target_counts[lab], replace=False)
    for cid in chosen:
        group = groups.get_group(cid)
        row = group.sample(n=1, random_state=SEED)
        selected_rows.append(row)
        used_clusters.add(cid)

df_synth_final = pd.concat(selected_rows, ignore_index=True)
df_synth_final = df_synth_final.sample(frac=1.0, random_state=SEED)

print("Final synthetic size:", len(df_synth_final))
print("Final label_3class counts:")
print(df_synth_final["label_3class"].value_counts().sort_index())


Target counts: {0: 900, 1: 700, 2: 400}
Final synthetic size: 2000
Final label_3class counts:
label_3class
0    900
1    700
2    400
Name: count, dtype: Int64


In [ ]:
# 7) Trim columns to the same schema as before

cols_final = [
    "subject",
    "body",
    "label",
    "label_3class",
    "label_binary",
    "label_name",
    "sender_domain",
    "url_domains",
    "date",
    "year",
    "source_dataset",
    "subject_len",
    "body_len",
]

df_synth_clean = df_synth_final[cols_final].copy()

print(df_synth_clean.info())
print("\nMissing values per column:")
print(df_synth_clean.isna().sum())
print("\nLabel_3class counts (final):")
print(df_synth_clean["label_3class"].value_counts().sort_index())


<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 1748 to 835
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject         2000 non-null   string        
 1   body            2000 non-null   string        
 2   label           2000 non-null   object        
 3   label_3class    2000 non-null   Int64         
 4   label_binary    2000 non-null   int64         
 5   label_name      2000 non-null   object        
 6   sender_domain   2000 non-null   object        
 7   url_domains     2000 non-null   object        
 8   date            2000 non-null   datetime64[ns]
 9   year            2000 non-null   int32         
 10  source_dataset  2000 non-null   object        
 11  subject_len     2000 non-null   Int64         
 12  body_len        2000 non-null   Int64         
dtypes: Int64(3), datetime64[ns](1), int32(1), int64(1), object(5), string(2)
memory usage: 216.8+ KB
None

Missing 

**Checking Synthetic Data:**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

body_text = df_synth_clean["body"].fillna("").astype(str)

vec = TfidfVectorizer(
    lowercase=True,
    max_df=0.8,
    min_df=5,
    ngram_range=(1, 2),
    stop_words="english",
    max_features=5000,
)
X = vec.fit_transform(body_text)

nn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=5)
nn.fit(X)
dist, idx = nn.kneighbors(X)
sim = 1 - dist

threshold = 0.95
pairs = []
n_rows = X.shape[0]
for i in range(n_rows):
    for j_idx, s in zip(idx[i, 1:], sim[i, 1:]):
        if j_idx == i:
            continue
        if s >= threshold:
            a, b = sorted((i, j_idx))
            pairs.append((a, b))

pairs = list(set(pairs))
print(f"Near-duplicate BODY pairs in FINAL synthetic set (sim >= {threshold}):", len(pairs))


Near-duplicate BODY pairs in FINAL synthetic set (sim >= 0.95): 10


In [ ]:
# ------------------------------------------------------------------
# 10) OPTIONAL: near-duplicate check on body-only (no changes to df_synth_clean)
# ------------------------------------------------------------------

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

body_text = df_synth_clean["body"].fillna("")

vectorizer_body = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words="english"
)
X_body = vectorizer_body.fit_transform(body_text)

nn_body = NearestNeighbors(
    n_neighbors=2,
    metric="cosine"
)
nn_body.fit(X_body)

distances_b, indices_b = nn_body.kneighbors(X_body)
similarities_b = 1 - distances_b[:, 1]

threshold_b = 0.99
near_dupe_mask_b = similarities_b >= threshold_b
n_near_dupes_b = near_dupe_mask_b.sum()

print(f"Rows with body-only similarity >= {threshold_b}: {n_near_dupes_b}")


Rows with body-only similarity >= 0.99: 0


In [ ]:
df_synth_clean.head()

,subject,body,label,label_3class,label_binary,label_name,sender_domain,url_domains,date,year,source_dataset,subject_len,body_len
1748,[Alibaba Notification] Inquiry from Emily Yan,Request for Quotation Trade Center Emily Yan I...,Phishing,2,1,phishing,mailsender.com,[],2025-06-03,2025,MergedAll_SynthPool,45,1890
680,"[Spambayes-checkins] spambayes setup.py,1.32,1...",Revision of /cvsroot/spambayes/spambayes In di...,Valid,0,0,ham_legit,users.sourceforge.net,[],2025-02-05,2025,MergedAll_SynthPool,53,906
1975,Immediate email termination alert for jose@mon...,"Dear User, Your email account is at risk of im...",Phishing,2,1,phishing,standardbank.co.za,[],2025-02-22,2025,MergedAll_SynthPool,53,367
318,[Bug 5814] uri PORN_4 regex incorrectly flags ...,http://issues.apache.org/SpamAssassin/show_bug...,Valid,0,0,ham_legit,issues.apache.org,[],2025-07-08,2025,MergedAll_SynthPool,77,1041
1483,LAST CHANCE: Claim Your Free Domain(s) and Wea...,LAST CHANCE: Claim Your Free Domain(s) and Wea...,Spam,1,0,spam,wealthyaffiliate.com,[],2025-08-03,2025,MergedAll_SynthPool,73,3064


In [ ]:
synth_path = os.path.join(PATH, "emails_synthetic.csv")
df_synth_clean.to_csv(synth_path, index=False)
print("Saved synthetic dataset to:", synth_path)


Saved synthetic dataset to: /content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/emails_synthetic.csv


In [ ]:
import pandas as pd

# Load your synthetic dataset
df_syn = pd.read_csv(PATH + "emails_synthetic.csv")  # change path if needed

# Count class labels
df_syn['label'].value_counts()


,count
label,
Valid,900
Spam,700
Phishing,400


In [ ]:
(df_syn['label'].value_counts(normalize=True) * 100).round(2)


,proportion
label,
Valid,45.0
Spam,35.0
Phishing,20.0


In [ ]:
dist = df_syn['label'].value_counts().rename("count")
dist_pct = (df_syn['label'].value_counts(normalize=True) * 100).round(2).rename("percent")

pd.concat([dist, dist_pct], axis=1)


,count,percent
label,,
Valid,900,45.0
Spam,700,35.0
Phishing,400,20.0


In [ ]:
df_syn.groupby(['source_dataset', 'label']).size().unstack(fill_value=0)


label,Phishing,Spam,Valid
source_dataset,,,
MergedAll_SynthPool,400,700,900


In [ ]:

PATH = "/content/drive/MyDrive/COSC 4371 - Security Analytics/Adversial Machine Learning - Phishing Email Classifier/Datasets/"

# Load datasets
before = pd.read_csv(os.path.join(PATH, "emails_before2010_balanced.csv"))
after = pd.read_csv(os.path.join(PATH, "emails_after2010_balanced.csv"))
synth = pd.read_csv(os.path.join(PATH, "emails_synthetic.csv"))

# Label names (same mapping as main notebook)
label_names = {0: "ham_legit", 1: "spam", 2: "phishing"}

def compute_counts(df, name):
    """Return a table of ham/spam/phishing counts for one dataset."""
    counts = (
        df.groupby("label_3class")
          .size()
          .reindex([0,1,2], fill_value=0)
          .rename(index=label_names)
          .to_frame(name)
    )
    return counts

# Compute for each dataset
counts_before = compute_counts(before, "before_2010")
counts_after = compute_counts(after, "after_2010")
counts_synth = compute_counts(synth, "synthetic")

# Display
print("\n=== Counts for PRE-2010 dataset ===")
print(counts_before)

print("\n=== Counts for POST-2010 dataset ===")
print(counts_after)

print("\n=== Counts for SYNTHETIC dataset ===")
print(counts_synth)



=== Counts for PRE-2010 dataset ===
              before_2010
label_3class             
ham_legit            1103
spam                  571
phishing              326

=== Counts for POST-2010 dataset ===
              after_2010
label_3class            
ham_legit             70
spam                 932
phishing             998

=== Counts for SYNTHETIC dataset ===
              synthetic
label_3class           
ham_legit           900
spam                700
phishing            400


#### **Individual Evaluation:**

To measure *performance*, I'll most likely use the standard classification report before & after TextAttack that includes...
- **Precision** - measures the accuracy of positive predictions by calculating $\frac{True~~Positives}{True~~Positives~+~False~~Fositives}$
- **Recall** - measures a model's ability to identify all relevant instances of a feature. $\frac{True~~Positives}{True~~Positives~+~False~~Negatives}$
- **F1 score** - This score is calculated as the harmonic mean of precision and recall. $F_1 = 2 \times \frac{precision~\times~recall}{precision~+~recall}$
- **Clean Accuracy** I'll measure how well my encoder-only transformer classifies the emails without TextAttack.


To measure *robustness* under TextAttack, I'll use...
- **Attack Success Rate (ASR)** - complement of adversarial accuracy (how often the attack flips to benign). $\frac{Successful~~attacks}{Total~\#~~attacks~~attempted}$
- **Robust Accuracy** - measure of a model's performance on inputs that have been intentionally perturbed or corrupted. $\frac{Initial~~Accuracy~-~After~attack~~Accuracy}{\#~of~~samples}$
- **Average tokens changed / pertubation rate** - As name mentions, $Average~~tokens~~changed = \frac{\Sigma^N_{i=1}\times C_i}{N}$ and $Pertubation rate = \frac{\Sigma^N_{i=1}\times \frac{C_i}{T_i}}{N} \times~100\%$, where $C_i$ be the number of tokens changed, $T_i$ be the number of tokens in original text, and $N$ is the number of text examples that were successfully pertubed.

#### Structured Learning (RDAP/DNS Lookups)

**Adaboost:**

**LightGBM:**

#### **Individual Evaluation:**

To measure *performance*, I'll most likely use the standard classification report before & after TextAttack that includes...
- **Precision** - measures the accuracy of positive predictions by calculating $\frac{True~~Positives}{True~~Positives~+~False~~Fositives}$
- **Recall** - measures a model's ability to identify all relevant instances of a feature. $\frac{True~~Positives}{True~~Positives~+~False~~Negatives}$
- **F1 score** - This score is calculated as the harmonic mean of precision and recall. $F_1 = 2 \times \frac{precision~\times~recall}{precision~+~recall}$
- **Clean Accuracy** I'll measure how well my encoder-only transformer classifies the emails without TextAttack.


To measure *robustness* under TextAttack, I'll use...
- **Attack Success Rate (ASR)** - complement of adversarial accuracy (how often the attack flips to benign). $\frac{Successful~~attacks}{Total~\#~~attacks~~attempted}$
- **Robust Accuracy** - measure of a model's performance on inputs that have been intentionally perturbed or corrupted. $\frac{Initial~~Accuracy~-~After-attack~~Accuracy}{\#~of~~samples}$
- **Average tokens changed / pertubation rate** - As name mentions, $Average~~tokens~~changed = \frac{\Sigma^N_{i=1}\times C_i}{N}$ and $Pertubation ~~rate = \frac{\Sigma^N_{i=1}\times \frac{C_i}{T_i}}{N} \times~100\%$, where $C_i$ be the number of tokens changed, $T_i$ be the number of tokens in original text, and $N$ is the number of text examples that were successfully pertubed.

#### One-Class Anomaly Detection:

**1-Class Support Vector Machine:**

**Isolation Forest:**

#### 2-Stage Pipeline

**Baseline Pipeline:**
- TF-IDF + Logistic Regression for the subject/body text.
- AdaBoost for the RDAP/DNS lookups.
- Evaluate performance (& robustness?) for both models individually.
- Then fuse Logistic Regression on [text_prob, tabs_prob] to create a 2-stage pipeline.
- Evaluate pipeline performance/robustness under TextAttack.

#### 3-Stage Pipline

**Deep Pipeline:**
- I'll either use a One-Class SVM or a Isolation Forest as a preliminarily model to tag "suspicious" emails. (Anomaly gate)
- Then I'll use DistilBert for the subject/body text.
- And LightGBM for the RDAP/DNS lookups.
- First, I'll evaluate each of these models individual as their own classifier. Evaluating their performance/robustness.
- Then, I'll use the 3 models to build a 3-stage pipeline, then evaluate the pipelines' performance against the baseline.

After comparing results of each pipeline, *if time allows me* I'll also deploy both models in real-time then compare not only performance or robustness, but also further metrics like latency, availability, reliability, & operational costs.

### Pipeline Evaluation:
Including all the normal metrics to observe individual performance/robustness, other additional metrics used to evalute both pipelines include...

To compare whole pipelines, not just single models:
- **Stage-1 & 2 pass-through (coverage):** % of emails reaching each stage.
- **Stage-1 recall at that coverage:** phishing emails caught by S1.
- **Stage-2 & 3 conditional metrics (on S1-passed set):** precission/recall/F1, the basic classification report.
- **End-to-end metrics:** After all stages have concluded on both pipelines, compare basic evvaluations.
- **Quality-vs-coverage curve:** Coverage is the % o how much email trafffic we are processing, while quality is determing how good are those final predictions. I'll compare the quality-vs-coverage of both models to show the trade-off of the two.

To check how similar or different the models classify emails:
   - **Double_fault:** A measure how often two models make the same mistake on the same samples. $$DF(A, B)= \frac{1}{N} \Sigma^N_{i=1} I(y_A(i) \neq y_i and y_B(i) \neq y_i)$$ where $y_i$ represents the true label for feature $i$, $y_A(i)~~ \& ~~y_B(i)$ represent the predicted labels by models A and B, and $I(condition)$ represent the indicator function. A low DF value represents the models fail on different examples while a high DF value represent the models fail on same samples.
   - **Inter-model Kl/JS Divergence:** A measure how different the predicted probability distributions are between models for the same samples. Unlike double-fault, this compares the *soft predictions*

To check <u>*transferability*</u>:
- **Cross-model ASR:** Craft on Model A, test on Pipeline B (andd vice-versa). Lower transfer $\rightarrow$ more diverse defenses.
- **Ensemble shielding:** (pipeline vs its components): ASR when atacking just the text branch vs full pipeline.


To check whether whether certain models are worth the extra cost (especially with the complexity difference):
   - **Training time:** How long does it take to train each model? How much for the pipeline as a whole?
   - **Latency (p50. p95):** Inference time per email
   - **Throughput:** Emails per second
   - **Resource use:** CPU/GPU load, memory

**Ask professor on input between ideas:**
1. The basis of my project idea will rely on two groups of features, subject/body text & RDAP/DNS info.
2. First, I will start by creating a simple encoder-only transformer model that will only first only use subject/body text as features.
3. Afterwards, I will then evaluate it's performance with and without TextAttack through simple measures like reporting clean accuracy, robust accuracy vs edit budget, and the attack success rate.

4. Then, I'll have the baseline model use subject/body text as a initial flagging of an email which will then pass the flagged records into another encoder-only transformer model.
5. Those same records that were passed from the baseline model will be sent to a second model that classifies the flagged emails through not only subject/body text but also RDAP and DNS lookups as well.
    - This idea is based on certain DNS/RDAP features that likely indicate a phishing attack such as...
        - No A/MX record
        - New/recently establish domain
        - Already expired registration date
        - Privacy/proxy registration indicates use of a privacy shield
        - Uses "addPeriod", "clientTransferProhibited", or "clientHold" EPP status codes.
            - "addPeriod" is a status code for a domain created within the recent 5 days.
            - legitimate domains usually have "clientTransferProhibited" to protect against domain hijacking
            - "clientHold" represents that the registrar put the domain on hold (DNS is disabled), thus likely from phishing attack.
        - Uses a registrar that is indicated as suspcious through a handmade list of what counts are suspcious registrars.
    - If I have time, I'd also like to see if I can utilize URLs by breaking them down into multiple lists of features. Not confirmed.
6. I'll then evaluate the performance on how well it handles itself with and without TextAttack; hopefully showing some progress
7. After all that, to connect this project to the idea of Computer Networks I plan to transition to evaluate the model's performance in real-time instead of just batch processing. This is important as due to my defenseive strategy being to implement RDAP/DNS lookup I plan that the initial evaluation will be poor to due to DDoS attacks from TextAttack from the massive amount of "fake edited" email entries that will cause a bottleneck due to my model using RDAP/DNS lookups.
8. Thus, I'll implement some (web caching?) implementation to prevent this as well..
Implement a basic model for a baseline -> use TextAttack and evaluate performance -> implement RDAP/DNS lookup as a defensive strategy -> implement and compare results.


Evalute results from batch processing -> implement real-time model -> evaluate RDAP/DNS query performance between baseline vs w/ TextAttack's additional inputs -> Think of query optimization ideas for defense -> evaluate RDAP/DNS query performance with query optimization


### Deploying models in real-time

#### Utilizing Text-Attack
**Objective:** Based on my current knowledge and research, one of they key drawbacks of using so many models is the amount of time it will take to train such models and also the amount of time TextAttack may take since it operates the inputs based on the outcome of the outputs. Thus, to combat this I also plan to use faster variation of TextAttack such as <u>**TextFooler**</u> and also customize such framework by editing certain parameters.

I plan to do this by...
- Changing the **MaxWordsPertubed** parameter, which influences the % of  words within the email TextFooler can edit.
- Changing the default **max candidates** parameters for TextFooler, which represents the # of max candidate keys per edit.
- Change the **UniversalSentenceEncoder** parameter, which how TextFooler customizes semantic similarity of the edits.

## Real-time Evaluation
In addition to all other metrics we considered, we will add a whole new variety of metrics to ensure our real-time pipeline performs well, especially underr TextAttack. Things to consider include...

Metrics to measure how the <u>**performance**</u> of the pipelines behave under live conditions:
 - **End-to-end latency (p50, p90, p95, p99):** Measuring latency throughout the entire pipeline.
 - **Stage latency breakdown:** Evaluating how latency changes (if it does) throughout different pipeline stages.
 - **Throughput:** Emails processed per sec/minute.
 - **Jitter (variance):** Std. deviation of latency per message
 - **Queue metrics:** Average queue length, waiting time
 - **Time-to-first-decision:** Average time from email arrival to email classification.
 - **Error rate:** $ off requests resulting in either a error, timmeout, or exception

Metrics to measure <u>**real-time data integration/lookup**<u/>, especially since we'll be querying RDAP/DNS live:
- **Query success rate:** Fraction of DNS/RDAP lookups returning valid results
- **Cache hit ratio:** % off lookups served from cachhe vs live query.
- **Async timeout rate:** % of lookups exceeding timeout threshold.
- **Stale-data fraction:** % of cached entries older than freshness threshold.
- **Dependency latency impact:** Difference in end-to-end latency when lookups are disabled vs enabled (idea unsure how to implement?)

Metrics to measure how <u>**robust**</u> or <u>**reliable**</u> the pipeline is under stress:
- **Failure recovery time (MTTR):** Time to recover from component crash or timeout.
- **Graceful degradationn success:** % off cases where system fall back correctly when a stage fails for some reason
- **Resource utilization stability:** CPU/GPU/memory variance under load spikes.

Metrics to measure **operational efficiency** (are the benefits worth the cost?):
- **Compute cost per 100 emails:** Aggregate CPU/GPU seconds $\times$ unit price.
- **Network cost per 100 emails:** Especially for external RDAP/DNS calls.

#### Combating DDoS attacks

**Objective**: If I have enough time to deploy a pipe-line in real time, I plan to install another a traffic flood generator (i.e <u>**Locust**</u> or <u>**asyncio Poisson**</u>) to simulate a DDoS attack onto my pipeline. Thus, in conjuction with TextAttack which will test my models robustness under adversial edits, asyncio Poisson wile help test my models robustness under adversial staffic. Since, my pipelines rely on live RDAP/DNS queries the more emails TextFooler makes $\rightarrow$ the more emails I'll have to classify $\rightarrow$ the more RDAP/DNS lookups I'll have to use $\rightarrow$ which leads to more RDAP/DNS queries $\rightarrow$ increased possibility of bottlenech due to the RDAP/DNS queries.

Thus, to prevent this I've listed various ideas I plan to test to combat this:

1) Reducing the flow of emails passing through my models.
    - **Rate-limiting:** I could put a cap on how many emails can be sent per second per sender or IP.
    - **Reputation-lists:** Could have a blacklist of definite suspicious domains/IPs that are prown to be used by phishing attacks. Or vice versa, a list of domain/IPs that guarantee somewhat legitimacy and have them skip stages?
    - **Sanity checks:** Could instantly drop emails that are missing subject/body text, or those with an abnormally long subject/body text.


2) Put hard limits on how much processing power or time my pipeline spends per email.
    - **Latency budget per request:** Each email has a maximum amount of processing time.
    - **Stage timeouts:** Each stage (text model, RDAP/DNS lookup) has its own smaller time cap.
    - **Lookup budget:** Could put a cap on the # of queries I use for RDAP/DNS lookups? If there are too many, I could either sample or truncate?


3) Keeping my system running and still giving reasonable results, despite any timeouts, missing data, trafffic, etc.
    - **Stage fallback:** If a slow or failing stage (i.e, RDAP lookups) doesn't return in time, I could skip it and rely on a harsher text-only prediction?
    - **Timeout + default behavior:** Give each stage a time limit. If an email request times out then we can default it to a phishing attack? Same if we have little info regardinng an emmail?


4) Prevent attackers from easily crafting emails that force a lot of extra computation per message.
    - **Limit input complexity:** I could put a limit on the maximum number of tokens used per email, or the # of domains/URLs used in an email. If they go over that limit, instant phishing classification?
    - **Homoglyph folding:** Essentially, modify tokenization to ensure that "google.com" and "ɢoogle.com" are treated the same?
    - **Whitespace collapsing:** Treat multiple spaces/tabs as only one space.
    - **Cache and reuse results:** Cache transfformer embeddings or RDAP responses for repated domains/phrases. Or reuse previous classifications on near-duplicates?
    - **Change model architecture:** If all else fails, I can always switch models between pipelines if one is too complex or less reliable than the other for the "ultimate" pipeline.


5) Make the RDAP/DNS stage more resilient so even if lookups spike it wont crash or stall.
   - **Use caching:** Keep a local cache of every domain referenced $\rightarrow$ RDAP/DNS results. Keep a "negative cache" ffor nonexistent dommains. Or set a *time-to-live (TTL)* for cache entries.
   - **Limit email lookups:** Put a hard limit on how many lookups the mmodel can do per mmessage. Choose *most suspicious-looking* domains first.
   - **Set timeouts and budgets:** Could give each query a deadline (e.g., 500 ms) or if the lookup doesn't respond skip it and default it to a phishing email?
   - **Batch and coalesce:** If multiple emails ask about the same domain then I could coalesce them into one lookup and share result?
   - **Use a local cursive DNS resolver or RDAP mirror?** ?

6) Develop my pipeline to detect a DDoS attack early.
    - **Baseline behavior:** First, I have to create a thresholds for normal pipeline network behavior like (latency p95 = 120 ms), (Cache hit <= 80%), etc.
    - **Automatic response based on alert:** After a alert signifies abnormal behavior, my model should import the various defenses I've explained above depending on what type of behavior is it.


## Risk Mitigation

---

**Vulnerabilities of my model:**
- Many email datasets are old (pre-2020), incomplete, or missing the required features I need to utilize the RDAP/DNS lookup idea. Also requires my DNS queries to be accurate with the most updated information or it may backffire.
- There's also the issue that my RDAP/DNS lookups may cause some bottelneck issues or even time out my model completely, especially in a realtime scenario if my assumption that TextAttack might try to DDoS my model is correct.
- In a realistic setting, domain features (like registrar country or age) change over time — a model trained today may degrade in performance in the fture especially if I don't ensure the most up-to-date informatioon.

**Flaws in project idea:**
- My ambition to implement my model both in batch-processing AND real-time and evaluate RDAP/DNS query performance seems a bit extra and may be implausible withinn the 6 week timeframe if I don't keep a consistent work ethic.
- With the amount of models I plan to use, it will take an extensive amount of time to train each one.
- Based off of my EDA of Nazario5, there are 9 other datasets I can use in Zenodo and if each one has the same distribution in email sample age than I'd probably get an average of around 100 email per dataset that are within the acceptable time range (2025 - 2022). Which will leave my total dataset with around 1000 records. This could be too low for my results to be reliable. I'll try to find a dataset but atm this one was the best I could find as not many datasets allow for the most recent email samples for security reasons.

**Bad Habits?**
- Like every college student, the #1 enemy of mine is procastination. If I do not ensure myself to consistently work on my project everyday I will fail.
- I also tend to "bite more than I can chew" or try out ambitious idea such as how I want to implement my model in real-time; however, since we only have 6 weeks it may be close to whether I can finish or not...
- I am working this project solo, as such when compared to others this may or may not be seen as a disadvantage.

## Logistics & Formatting

---
**Objective:** This is the worst-case timeline I've created assuming I do every last minute. I do hope to finish most of these phases earlier than planned; however, these are the dates I will finish at the LATEST.

Phase 1 — Dataset & Setup (Oct. 14 - Oct. 20)
- Combine all datasets into one. (Oct. 16)
- Filter to 2020-2025 emails. (Oct. 16)
- Parse sender/receiver/URL domains. (Oct. 17)
- Build script for RDAP/DNS lookups and caching. (Oct. 18)
- Draft feature schema (Has_MX, Has_SPF, Registrar_country, Is_new_domain, etc.). (Oct. 19)

Phase 2 - Stage 1 Text Model (Oct. 21 - Oct. 27)
- Implement baseline TF-IDF + LogReg. (Oct. 22)
- Fine-tune DistilBERT for text classification. (Oct. 24)
- Evaluate precision/recall/F1. (Oct. 25)
- Export predicted probabilities for downstream fusion. (Oct. 27)

Phase 3 — Stage 2 RDAP/DNS Model (Oct. 28 - Nov. 3)
- Compute all RDAP/DNS features with async queries. (Oct. 28)
- Train LightGBM and AdaBoost models. (Oct. 30)
- Analyze feature importance. (Nov. 1)
- Store inference pipeline (input → lookup → vector). (Nov. 2)

Phase 4 — Stage 0 / Stage 3 Anomaly Gate (Nov. 3 - Nov. 10)
- Build One-Class SVM or Isolation Forest on clean text embeddings. (Nov. 4)
- Calibrate threshold for coverage (~80–95%). (Nov. 5)
- Log stage pass-through statistics. (Nov. 7)
- Evaluate individual metrics (Nov. 9)

Phase 5 - Full 3-Stage Pipeline (Nov. 10 - Nov. 19)
- Chain all stages: Anomaly → DistilBERT → LightGBM. (Nov. 13)
- Create per-stage metrics (quality, coverage, latency). (Nov. 16)
- Plot quality-vs-coverage curves. (Nov. 17)
- Save modular code for swapping models. (Nov. 18)

Phase 5.5 - Baseline 2-Stage Pipeline (Nov. 10 - Nov. 20) *Optional*
- Build TF-IDF + AdaBoost + meta-LR fusion. (Nov. 14)
- Create per-stage metrics. (Nov. 15)
- Plot quality-vs-coverage curves. (Nov. 16)
- Compare clean and perturbed performance to 3-stage pipeline. (Nov. 17)
- Record compute/time cost. (Nov. 18)

Phase 6 - Adversarial Evaluation (Nov. 20 - Nov. 22)
- Configure TextAttack/TextFooler budgets (0–15% edits). (Nov. 18)
- Run attacks on Stage 1 & full pipeline. (Nov. 19)
- Compute ASR, robust F1, token-edit histograms. (Nov. 20)
- Compare transfer attacks between pipelines. (Nov. 22)

Phase 7 - Real-Time Simulation & DDoS Tests (Nov. 23 - Nov. 26)
- Implement async or threaded pipeline runner. (Nov. 23)
- Simulate load (Poisson arrival, flood test). (Nov. 25)
- Collect latency histograms (p50/p95/p99). (Nov. 25)
- Add caching, rate-limit, timeout, fallback defenses. (Nov. 26)
- Measure degradation curves. (Nov. 26)

Phase 8 - Analysis & Visualization (Nov. 27 - Nov. 28)
- Aggregate all metrics into tables. (Nov. 27)
- Draw plots: ROC, quality-coverage, ASR vs budget, latency CDF. (Nov. 28)
- Summarize findings: robustness vs cost trade-off. (Nov. 29)

Phase 9 - Final Report & Presentation (Nov. 28 - Dec. 1)
- Write Abstract, Methodology, Results, Discussion. (Nov. 29)
- Include diagrams of pipelines, defense flow, and metrics. (Nov. 30)
- Proofread and finalize visuals. (Dec. 1)
- Prepare slides/demo notebook. (Dec. 1)